In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "colabcctbx=snips.ipynb",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    },
    "gpuClass": "standard"
  },
  "cells": [
    {
      "cell_type": "markdown",
      "source": [
        "# Install the snippet notebook\n",
        "\n",
        "Store the snippet notebook on your Google Drive. Then copy the updated URL to \n",
        "Tools --> Settings --> Site --> Custom snippet notebook URL. Then when you open \n",
        "a new notebook, the snippets will be available.\n",
        "\n",
        "1.   File --> Save Copy in Drive\n",
        "2.   Tools --> Settings --> Site --> Custom snippet notebook URL\n"
      ],
      "metadata": {
        "id": "jda5IgVWoB0v"
      }
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ft3z-TiIKKmM"
      },
      "outputs": [],
      "source": [
        "# Install condacolab so that conda packages can be used\n",
        "# https://github.com/conda-incubator/condacolab\n",
        "#\n",
        "# Important notes\n",
        "# https://github.com/conda-incubator/condacolab#shortcomings\n",
        "# 1) The kernel will automatically be restarted and show an error message about\n",
        "#    a crash (\"Your session crashed for an unknown reason\").\n",
        "#    The error can be ignored.\n",
        "# 2) Only the \"base\" environment is available, so do not create a separate \n",
        "#    environment for pacakges.\n",
        "from IPython.utils import io\n",
        "import tqdm.notebook\n",
        "\n",
        "total = 50\n",
        "\n",
        "with tqdm.notebook.tqdm(total=total) as pbar:\n",
        "    with io.capture_output() as captured:\n",
        "\n",
        "        !pip install -q condacolab\n",
        "\n",
        "        pbar.update(10)\n",
        "\n",
        "        import condacolab\n",
        "        condacolab.install()\n",
        "\n",
        "        pbar.update(40)\n",
        "\n",
        "# An error message will appear at the bottom of the notebook about \n",
        "# your session crashing for an unknown reason. Click on the X and \n",
        "# run the next cell.     "
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "\n",
        "with tqdm.notebook.tqdm(total=total) as pbar:\n",
        "    with io.capture_output() as captured:\n",
        "\n",
        "        # Install cctbx-base\n",
        "        # Wait until the error message appears near the bottom of the notebook before\n",
        "        # proceeding.\n",
        "\n",
        "        # By default, condacolab installs both conda and mamba. Generally, mamba will\n",
        "        # be faster when resolving dependencies.\n",
        "        !mamba install -q cctbx-base\n",
        "        pbar.update(10)\n",
        "        # conda installs ${CONDA_PREFIX}/share/cctbx into /usr/local instead of /usr\n",
        "        # Make a copy to avoid errors\n",
        "        !cp -af /usr/local/share/cctbx /usr/share/\n",
        "\n",
        "        # sys.path does not have some directories with libraries, so add them here\n",
        "        import sys\n",
        "        for d in ['/usr/local/lib/python3.7/lib-dynload', '/usr/local/lib']:\n",
        "          if d not in sys.path:\n",
        "            sys.path.insert(0, d)\n",
        "        pbar.update(10)\n",
        "        # final check\n",
        "        import os\n",
        "        if os.path.isdir('/usr/local/share/cctbx') \\\n",
        "          and '/usr/local/lib/python3.7/lib-dynload' in sys.path \\\n",
        "          and '/usr/local/lib' in sys.path:\n",
        "          print('Finished installing cctbx-base')\n",
        "        else:\n",
        "          raise RuntimeError('There was an error fixing up the installation of cctbx-base')\n",
        "        pbar.update(30)"
      ],
      "metadata": {
        "id": "TFtfP--2Kdnn"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# Start running cctbx code\n",
        "# Thanks to Georg Mlynek for this example!\n",
        "from cctbx import crystal, miller\n",
        "\n",
        "crystal_symmetry = crystal.symmetry((10, 20, 30, 90, 90, 90), 'P212121')\n",
        "ms = miller.build_set(\n",
        "    crystal_symmetry=crystal_symmetry,\n",
        "    anomalous_flag=True,\n",
        "    d_min=2,\n",
        "    d_max=10\n",
        ")\n",
        "crystal_symmetry.show_summary()\n",
        "print('Resolution range', ms.d_max_min())\n",
        "print('Number of reflections', ms.indices().size())"
      ],
      "metadata": {
        "id": "jug2vYAgLpQF"
      },
      "execution_count": null,
      "outputs": []
    }
  ]
},
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "208168398ac9"
      },
      "source": [
        "##millerMtzImports\n",
        "\n",
        "Read a mtz file into a miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "62d3cc6895be"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_file = any_reflection_file('3hz7.mtz')\n",
      "miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8b4b6d33116b"
      },
      "source": [
        "##millerArrayLabels\n",
        "\n",
        "Print column labels in a Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "83a47537bccf"
      },
      "source": [
      "[print('Miller Array %s: %s' % (i, miller_array.info().labels)) for i, miller_array in list(enumerate(miller_arrays))[:2]]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "bc12eecc7a0"
      },
      "source": [
        "##millerArrayWavelengths\n",
        "\n",
        "Print wavelengths of each miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d2be19c12c18"
      },
      "source": [
      "[print('Miller Array %s: %s' % (i, miller_array.info().wavelength)) for i, miller_array in list(enumerate(miller_arrays))]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "268edb6e0128"
      },
      "source": [
        "##millerArraySources\n",
        "\n",
        "Print the source of each miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "166a05057972"
      },
      "source": [
      "print('Miller Array %s: %s' % (i, miller_array.info().source)) for i, miller_array in list(enumerate(miller_arrays))]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e0ca4156699c"
      },
      "source": [
        "##millerArrayLengths\n",
        "\n",
        "Print length of miller arrays (i.e., the number of datasets in a mtz file).\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "faae0e7066a6"
      },
      "source": [
      "len(miller_arrays)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "84aefb22c1aa"
      },
      "source": [
        "##millerArraySymmetry\n",
        "\n",
        "Print the crystal symmetry of each miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d823a99d4213"
      },
      "source": [
      "[print('Miller Array %s: %s' % (i, miller_array.info().crystal_symmetry_from_file)) for i, miller_array in list(enumerate(miller_arrays))]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e37b51f35931"
      },
      "source": [
        "##millerArrayHKLs\n",
        "\n",
        "Print all of the miller indices for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "68be1765ce64"
      },
      "source": [
      "[print(hkl) for hkl in miller_arrays[0].indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "fc8428423e53"
      },
      "source": [
        "##millerArrayMethods\n",
        "\n",
        "Print the available methods for the Miller class.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "ec890efbd83f"
      },
      "source": [
      "dir(miller_arrays[0])\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a4a93b6b0a8c"
      },
      "source": [
        "##millerArraydstar\n",
        "\n",
        "Return the resolution range in d* in a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "41c40922a9d3"
      },
      "source": [
      "miller_arrays[0].min_max_d_star_sq()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c8053e6577ef"
      },
      "source": [
        "##millerArrayDminDmax\n",
        "\n",
        "Return the resolution range in Angstroms for a Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "9a5fce31c3a3"
      },
      "source": [
      "miller_arrays[0].d_max_min()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "893e5da08192"
      },
      "source": [
        "##millerArrayIsigma\n",
        "\n",
        "Return the I/sig overall for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "f3fa3a7fe014"
      },
      "source": [
      "miller_arrays[0].i_over_sig_i()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "5caf7351d4"
      },
      "source": [
        "##millerArrayCC\n",
        "\n",
        "Return CC one-half sigma tau for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "401407a262d2"
      },
      "source": [
      "miller_arrays[0].cc_one_half_sigma_tau()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "192482022257"
      },
      "source": [
        "##millerArrayCConeHalf\n",
        "\n",
        "Return CC one-half for a given Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "cf41e7e951f0"
      },
      "source": [
      "miller_arrays[0].cc_one_half()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "6a03c2489db6"
      },
      "source": [
        "##millerArrayBijvoetRatios\n",
        "\n",
        "Print the Bijvoet ratios in a specified Miller array.  May have to average by bin first.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "350f95424ead"
      },
      "source": [
      "[print(i) for i in miller_arrays[0].bijvoet_ratios()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d85fc84cb1a8"
      },
      "source": [
        "##millerArrayMeasurability\n",
        "\n",
        "Return the `measurability` of the anomalous signal in a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "6b27fa87c72"
      },
      "source": [
      "miller_arrays[0].measurability()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "35b3248a77eb"
      },
      "source": [
        "##millerArrayAnomalousSignal\n",
        "\n",
        "Return the anomalous signal in a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "17755cd1bfb1"
      },
      "source": [
      "miller_arrays[0].anomalous_signal()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a41d1efbd2f8"
      },
      "source": [
        "##millerArrayComprehensiveSummary\n",
        "\n",
        "Show comprehensive summary for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "a47b4a02b5e6"
      },
      "source": [
      "miller_arrays[0].show_comprehensive_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e5f479dcd50b"
      },
      "source": [
        "##millerArrayCountBijvoetPairs\n",
        "\n",
        "Show number of bijvoet pairs for a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "841f26aed1c5"
      },
      "source": [
      "miller_arrays[0].n_bijvoet_pairs()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a47efa001cf6"
      },
      "source": [
        "##millerArrayWilsonRatio\n",
        "\n",
        "Show wilson ratio of miller array for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "b088ca566917"
      },
      "source": [
      "miller_arrays[0].wilson_ratio()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "3c8d4fed4bc7"
      },
      "source": [
        "##millerArrayUnpackIpIn\n",
        "\n",
        "Unpack into I(+) and I(-) for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "446eb66d531f"
      },
      "source": [
      "Iobs = miller_arrays[0]\n",
      "i_plus, i_minus = Iobs.hemispheres_acentrics()\n",
      "ipd = i_plus.data()\n",
      "ip=list(ipd)\n",
      "imd = i_minus.data()\n",
      "im = list(imd)\n",
      "len(im)\n",
      "Iobs.show_summary()\n",
      "print(Iobs.info())\n",
      "print(Iobs.observation_type())\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4df81701ca5d"
      },
      "source": [
        "##millerArrayPrintSelectRows\n",
        "\n",
        "Print five rows of the Iobs for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2ca578e6442c"
      },
      "source": [
      "list(Iobs[100:105])\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "34b8af1bc1de"
      },
      "source": [
        "##millerArrayExtractIntensities\n",
        "\n",
        "Extract just the intensities for a give Miller array and print ten rows of them.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "ae86a88233df"
      },
      "source": [
      "Iobs = miller_arrays[0]\n",
      "iobsdata = Iobs.data()\n",
      "list(iobsdata[100:110])\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ea29c0120f25"
      },
      "source": [
        "##millerArrayPrintntensities\n",
        "\n",
        "Print all of the intensities for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "b7244e71c098"
      },
      "source": [
      "[print(hkl) for hkl in miller_arrays[1].data()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8c2af4eb6bd"
      },
      "source": [
        "##millerArrayconvert2mtz\n",
        "\n",
        "Convert the miller array into a mtz_dataset and write out as a mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "89c5a531b296"
      },
      "source": [
      "# Convert the miller array into a mtz_dataset and write out as a mtz file.\n",
      "mtz_dataset = Iobs.as_mtz_dataset(column_root_label='I')\n",
      "mtz_dataset.mtz_object().write('3hz7intensities.mtz')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c96ef65d7e72"
      },
      "source": [
        "##millerArrayReadMtz\n",
        "\n",
        "Read in the mtz file and print its column labels as a sanity check.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "caf513cc7bbb"
      },
      "source": [
      "mtz_filename2 = '3hz7intensities.mtz'\n",
      "mtz_file2 = mtz.object(mtz_filename2)\n",
      "mtz_file2.column_labels()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "96248422e5c7"
      },
      "source": [
        "##millerArray2Dictionary\n",
        "\n",
        "Set up the arrays as dictionaries\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "b1e1e45d0b04"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='3nd4.mtz')\n",
      "# Only works with mtz.object. \n",
      "# Does not work if mtz is read in with iotbx.file_reader.\n",
      "miller_arrays_dict = mtz_obj.as_miller_arrays_dict()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "35cce1a44022"
      },
      "source": [
        "##millerArrayDictionaryKeys\n",
        "\n",
        "Print the miller keys() of a miller dictionary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "48d1178aeef7"
      },
      "source": [
      "miller_arrays_dict.keys()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "2c7b88499527"
      },
      "source": [
        "##millerArrayDictPrintColumns\n",
        "\n",
        "Print the column labels of Miller dictionary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d40ed1e99264"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='/Users/blaine/3nd4.mtz')\n",
      "# Only works with mtz.object. Does not work if mtz is read in with iotbx.file_reader.\n",
      "miller_arrays_dict = mtz_obj.as_miller_arrays_dict()\n",
      "[print(f'Column label: {key[2]')  for key in miller_arrays_dict.keys()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "55632319e8e9"
      },
      "source": [
        "##millerArrayBuildFromUnitCell\n",
        "\n",
        "Peter Zwart's code for generating the indices for unit cell of given symmetry and dimensions. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "56b644a89b93"
      },
      "source": [
      "from cctbx import miller\n",
      "import cctbx\n",
      "from cctbx import crystal\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='Fd-3m',\n",
      "        unit_cell=(5.4307,5.4307,5.4307,90.00,90.0,90.00)),\n",
      "    anomalous_flag=True,\n",
      "    d_min=0.4)\n",
      "\n",
      "[print(hkl) for hkl in ms.indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "b58a3485f66e"
      },
      "source": [
        "##millerArrayMapToASU\n",
        "\n",
        "Map generated reflections to the asu and print.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "886d75ddeae8"
      },
      "source": [
      "from cctbx import miller\n",
      "import cctbx\n",
      "from cctbx import crystal\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='Fd-3m',\n",
      "        unit_cell=(5.4307,5.4307,5.4307,90.00,90.0,90.00)),\n",
      "    anomalous_flag=True,\n",
      "    d_min=0.4)\n",
      "\n",
      "msu = ms.map_to_asu()\n",
      "[print(hkl2) for hkl2 in msu.indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4ffecd39637d"
      },
      "source": [
        "##millerArrayPrintSummary\n",
        "\n",
        "Read mtz file into a miller array and print summary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "4d9212194b1f"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_in = any_reflection_file(file_name='3nd4.mtz')\n",
      "miller_arrays = hkl_in.as_miller_arrays()\n",
      "f_obs = miller_arrays[0]\n",
      "f_obs.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c53d3e16d746"
      },
      "source": [
        "##mtzObjectSummary\n",
        "\n",
        "Read mtz file into a mtz object and print summary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "9e0a15c8c416"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='/Users/blaine/3nd4.mtz')\n",
      "mtz_obj.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ff1d6fecee92"
      },
      "source": [
        "##millerArrayFromMtz\n",
        "\n",
        "Read mtz file into a Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c33a55f687e"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='3nd4.mtz')\n",
      "miller_arrays = mtz_obj.as_miller_arrays()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "52396db4729f"
      },
      "source": [
        "##millerArrayTruncate\n",
        "\n",
        "Read mtz file into a Miller array, truncate, and print summary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "dddbc23278dc"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='3nd4.mtz')\n",
      "miller_arrays = mtz_obj.as_miller_arrays()\n",
      "miller_array_truncated = miller_arrays[0].resolution_filter(d_min=2, d_max=5)\n",
      "print(miller_array_truncated)\n",
      "miller_array_truncated.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "1b333a322347"
      },
      "source": [
        "##millerArrayDictColumnLabels\n",
        "\n",
        "Print column labels of a Miller array dictionary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5ea895b2ecdd"
      },
      "source": [
      "[print(f'Column label: {key[2]')  for key in miller_arrays_dict.keys()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "78decee0abad"
      },
      "source": [
        "##condaInstall\n",
        "\n",
        "The conda commands to install cctbx with the jupyter notebook, pandas, and xarray.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "927349abbab4"
      },
      "source": [
      "conda create --name cctbx37 python=3.7\n",
      "conda activate cctbx37\n",
      "conda install -c conda-forge cctbx-base jupyter pandas xarray\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "6e130c16ddb2"
      },
      "source": [
        "##condaRemove\n",
        "\n",
        "The conda commands to remove cctbx37 env.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2dcbcb90617e"
      },
      "source": [
      "conda remove --name cctbx37"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d24ce6413d08"
      },
      "source": [
        "##fetchPDB\n",
        "\n",
        "Fetch pdb file from RCSB in PDB format.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "74491a5d3935"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='pdb', mirror='rcsb', format='pdb', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "1615ebe8d95f"
      },
      "source": [
        "##updateCLT\n",
        "\n",
        "Update the command line tools for Xcode on Mac OS X. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d44b18ba1982"
      },
      "source": [
      "sudo touch /tmp/.com.apple.dt.CommandLineTools.installondemand.in-progress\n",
      "softwareupdate -l\n",
      "# Update command line tools via software update.\n",
      "sudo rm /tmp/.com.apple.dt.CommandLineTools.installondemand.in-progress"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "fdcb6594ccdd"
      },
      "source": [
        "##extractCrystalSymmetry\n",
        "\n",
        "Extract crystal symmetry from mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c8132818a43b"
      },
      "source": [
      "from __future__ import absolute_import, division, print_function\n",
      "from iotbx import mtz\n",
      "from cctbx import crystal\n",
      "\n",
      "def extract_from(file_name):\n",
      "  mtz_object = mtz.object(file_name=file_name)\n",
      "  assert mtz_object.n_symmetry_matrices() > 0\n",
      "  return mtz_object.crystals()[0].crystal_symmetry()\n",
      "  \n",
      " extract_from(file_name='3nd4.mtz')\n",
      " $0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a442e31a37e5"
      },
      "source": [
        "##plotDstarsLogMeans\n",
        "\n",
        "Generate the list of dstars and logMeans as lists for plotting by matplotlib.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5e0a6360d081"
      },
      "source": [
      "\"\"\"Generate the list of dstars and logMeans as lists\n",
      "for plotting by matplotlib.\"\"\"\n",
      "\n",
      "used = list(binner.range_used())\n",
      "selections = [binner.selection(i) for i in used]\n",
      "\n",
      "# make means of the intensities by bin\n",
      "means = [Iobs.select(sel).mean() for sel in selections]\n",
      "from math import log\n",
      "lnmeans = [log(y) for y in means]\n",
      "\n",
      "# meansBR = [Iobs.bijvoet_ratios().select(sel).mean() for sel in selections]\n",
      "\n",
      "# make d_centers\n",
      "d_star_power = 1.618034\n",
      "centers = binner.bin_centers(d_star_power)\n",
      "d_centers = list(centers**(-1 / d_star_power))\n",
      "\n",
      "%matplotlib inline\n",
      "import matplotlib.pyplot as plt\n",
      "import matplotlib as mpl\n",
      "mpl.rcParams['savefig.dpi'] = 600\n",
      "mpl.rcParams['figure.dpi'] = 600\n",
      "\n",
      "fig, ax = plt.subplots(figsize=[3.25, 2.])\n",
      "ax.scatter(d_centers,lnmeans,c='k',alpha=0.3,s=5.5)\n",
      "\n",
      "ax.set_xlim(8, 1.5) # decreasing time\n",
      "ax.set_xlabel(r'$d^*$ in $\AA$',fontsize=12)\n",
      "ax.set_ylabel('ln(I)',fontsize=12)\n",
      "plt.xticks(fontsize=12)\n",
      "plt.yticks(fontsize=12)\n",
      "ax.grid(False)\n",
      "plt.savefig('3hz7iobsvsdstar.pdf',bbox_inches='tight')\n",
      "plt.show()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c4a620720987"
      },
      "source": [
        "##plotDstarsMeasurability\n",
        "\n",
        "Generate the list of dstars and measurability as lists for plotting by matplotlib.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "29208760aa3a"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_file = any_reflection_file('3hz7.mtz')\n",
      "miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)\n",
      "\n",
      "Iobs = miller_arrays[1]\n",
      "# Set up the bins\n",
      "n_bins = 50\n",
      "binner = Iobs.setup_binner(n_bins=n_bins)\n",
      "# binner.show_summary()\n",
      "used = list(binner.range_used())\n",
      "selections = [binner.selection(i) for i in used]\n",
      "\n",
      "# make d_centers for the x-axis\n",
      "d_star_power = 1.618034\n",
      "centers = binner.bin_centers(d_star_power)\n",
      "d_centers = list(centers**(-1 / d_star_power))\n",
      "\n",
      "# make list of the measurabilities by resolution bin\n",
      "meas = [Iobs.select(sel).measurability() for sel in selections]\n",
      "\n",
      "%matplotlib inline\n",
      "import matplotlib.pyplot as plt\n",
      "import matplotlib as mpl\n",
      "mpl.rcParams['savefig.dpi'] = 600\n",
      "mpl.rcParams['figure.dpi'] = 600\n",
      "\n",
      "fig, ax = plt.subplots(figsize=[3.25, 2.])\n",
      "ax.scatter(d_centers,lnmeans,c='k',alpha=0.3,s=5.5)\n",
      "\n",
      "ax.set_xlim(8, 1.5) # decreasing time\n",
      "ax.set_xlabel(r'$d^*$ in $\AA$',fontsize=12)\n",
      "ax.set_ylabel('ln(I)',fontsize=12)\n",
      "plt.xticks(fontsize=12)\n",
      "plt.yticks(fontsize=12)\n",
      "ax.grid(False)\n",
      "plt.savefig('3hz7measureability.pdf',bbox_inches='tight')\n",
      "plt.show()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "aadb5199ff9a"
      },
      "source": [
        "##computeAllMillerIndices\n",
        "\n",
        "Compute all possible Miller indices.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "bfcc5fc9bc1"
      },
      "source": [
      "from cctbx import miller\n",
      "\n",
      "def generate_reflection_indices(uc, dmin):\n",
      "    maxh, maxk, maxl = uc.max_miller_indices(dmin)\n",
      "\n",
      "    indices = []\n",
      "\n",
      "    for h in range(-maxh, maxh + 1):\n",
      "        for k in range(-maxk, maxk + 1):\n",
      "            for l in range(-maxl, maxl + 1):\n",
      "                if h == 0 and k == 0 and l == 0:\n",
      "                    continue\n",
      "                if uc.d((h, k, l)) < dmin:\n",
      "                    continue\n",
      "        indices.append((h, k, l))\n",
      "    return indices\n",
      "    \n",
      "uc=(5.4307,5.4307,5.4307,90.00,90.0,90.00)\n",
      "dmin=1.0\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4fc0ba1f78a2"
      },
      "source": [
        "##computeAllMillerIndicesASU\n",
        "\n",
        "Compute all possible Miller indices in the ASU.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "76e2cde17500"
      },
      "source": [
      "from cctbx import miller\n",
      "import cctbx\n",
      "from cctbx import crystal\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='Fd-3m',\n",
      "        unit_cell=('5.4307,5.4307,5.4307,90.00,90.0,90.00') ),\n",
      "    anomalous_flag=False,\n",
      "    d_min=0.4)\n",
      "\n",
      "for hkl in ms.indices():\n",
      "    print(hkl)\n",
      "\n",
      "# map the reflections to the asu and print\n",
      "  \n",
      "msu = ms.map_to_asu()\n",
      "[print(hkl2) for hkl2 in msu.indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "39af28d53c59"
      },
      "source": [
        "##computeAllMillerIndicesUnitCell\n",
        "\n",
        "Build miller indices given unit cell and resolution limit.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7b53fc216787"
      },
      "source": [
      "from cctbx import crystal\n",
      "from cctbx import miller\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='P4',\n",
      "        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),\n",
      "    anomalous_flag=False,\n",
      "    d_min=1.4)\n",
      "msu = ms.map_to_asu()\n",
      "[print(hkl) for hkl in msu.indices()]\n",
      "print(msu.show_comprehensive_summary())\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4dac871a712d"
      },
      "source": [
        "##extractReflectionMtzFile\n",
        "\n",
        "Extract the reflections from a mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d9e90a70be34"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz')\n",
      "mtz_obj.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "b986eaa9b6ec"
      },
      "source": [
        "##extractReflectionInShell\n",
        "\n",
        "Extract the reflections in a shell.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "cf8e647451e0"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='2V89.mtz')\n",
      "miller_arrays = mtz_obj.as_miller_arrays()\n",
      "for miller_array in miller_arrays:\n",
      "    miller_array_truncated = miller_array.resolution_filter(d_min=2, d_max=5)\n",
      "print(miller_array_truncated)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a9e08c396c23"
      },
      "source": [
        "##plotRfactorResolutionBin\n",
        "\n",
        "Read in a phenix.refine mtz file. It plots the work and free R-factors by resolution bin.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "956cb3fcc790"
      },
      "source": [
      "#!/usr/bin/env python\n",
      "# coding: utf-8\n",
      "\"\"\" \n",
      "This script reads in a phenix.refine mtz file.\n",
      "It plots the R-factor by resolution bin.\n",
      "The plots are made with matplotlib using miller arrays.\n",
      "It also plots the correlation coefficients.\n",
      "The plots were made with matplotlib.\n",
      "\n",
      "This script was adapted from an example script in iotbx:  \n",
      "\n",
      "Source:  https://github.com/cctbx/cctbx_project/blob/master/\n",
      "iotbx/examples/recalculate_phenix_refine_r_factors.py\n",
      "\"\"\"\n",
      "\n",
      "\n",
      "# get_ipython().run_line_magic('matplotlib', 'inline')\n",
      "\n",
      "from __future__ import absolute_import, division, print_function\n",
      "from iotbx.reflection_file_utils import get_r_free_flags_scores\n",
      "from iotbx.file_reader import any_file\n",
      "import matplotlib\n",
      "import matplotlib.pyplot as plt\n",
      "\n",
      "\n",
      "\n",
      "def compute_r_factors(fobs, fmodel, flags):\n",
      "  fmodel, fobs = fmodel.common_sets(other=fobs)\n",
      "  fmodel, flags = fmodel.common_sets(other=flags)\n",
      "  fc_work = fmodel.select(~(flags.data()))\n",
      "  fo_work = fobs.select(~(flags.data()))\n",
      "  fc_test = fmodel.select(flags.data())\n",
      "  fo_test = fobs.select(flags.data())\n",
      "  r_work = fo_work.r1_factor(fc_work)\n",
      "  r_free = fo_test.r1_factor(fc_test)\n",
      "  \n",
      "  print('r_work = %.4f' % r_work)\n",
      "  print('r_free = %.4f' % r_free)\n",
      "  print('')\n",
      "\n",
      "  binner = flags.setup_binner(n_bins=20)\n",
      "  d_star_power = 1.618034\n",
      "  centers = binner.bin_centers(d_star_power)\n",
      "  d_centers = list(centers**(-1 / d_star_power))\n",
      "#   for i in d_centers:\n",
      "#     print(i)\n",
      "    \n",
      "  fo_work.use_binning_of(flags)\n",
      "  fc_work.use_binner_of(fo_work)\n",
      "  fo_test.use_binning_of(fo_work)\n",
      "  fc_test.use_binning_of(fo_work)\n",
      "\n",
      "  r_work_list = []\n",
      "  r_free_list = []\n",
      "  cc_work_list = []\n",
      "  cc_free_list = []\n",
      "  for i_bin in fo_work.binner().range_all():\n",
      "    sel_work = fo_work.binner().selection(i_bin)\n",
      "    sel_test = fo_test.binner().selection(i_bin)\n",
      "    fo_work_bin = fo_work.select(sel_work)\n",
      "    fc_work_bin = fc_work.select(sel_work)\n",
      "    fo_test_bin = fo_test.select(sel_test)\n",
      "    fc_test_bin = fc_test.select(sel_test)\n",
      "    if fc_test_bin.size() == 0 : continue\n",
      "        \n",
      "    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_work_list.append(r_work_bin)\n",
      "    \n",
      "    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_free_list.append(r_free_bin)\n",
      "    \n",
      "    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()\n",
      "    cc_work_list.append(cc_work_bin)\n",
      "    \n",
      "    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()\n",
      "    cc_free_list.append(cc_free_bin)\n",
      "    \n",
      "    legend = flags.binner().bin_legend(i_bin, show_counts=False)\n",
      "    print('%s  %8d %8d  %.4f %.4f  %.3f %.3f' % (legend, fo_work_bin.size(),\n",
      "      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))\n",
      "    \n",
      "  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list\n",
      "\n",
      "\n",
      "def plot_r_factors(d_centers, r_work_list, r_free_list):\n",
      "  plt.scatter(d_centers, r_work_list, label=r'$\mathit{R_{work$')\n",
      "  plt.scatter(d_centers, r_free_list, label=r'$\mathit{R_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'R-factor (%)')\n",
      "  plt.legend(loc='upper right')\n",
      "  plt.savefig('Rs.pdf')\n",
      "  plt.close()\n",
      "\n",
      "\n",
      "def plot_cc(d_centers, cc_work_list, cc_free_list):\n",
      "  plt.scatter(d_centers, cc_work_list, label=r'$\mathit{CC_{work$')\n",
      "  plt.scatter(d_centers, cc_free_list, label=r'$\mathit{CC_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'Correlation Coefficeint Fo vs Fc (%)')\n",
      "  plt.legend(loc='lower right')\n",
      "  plt.savefig('CCs.pdf')\n",
      "\n",
      "\n",
      "def run(input_mtz):\n",
      "  mtz_in = any_file(input_mtz)\n",
      "  ma = mtz_in.file_server.miller_arrays\n",
      "  flags = fmodel = fobs = None\n",
      "  # select the output arrays from phenix.refine.  This could easily be modified\n",
      "  # to handle MTZ files from other programs.\n",
      "  for array in ma :\n",
      "    labels = array.info().label_string()\n",
      "    if labels.startswith('R-free-flags'):\n",
      "      flags = array\n",
      "    elif labels.startswith('F-model'):\n",
      "      fmodel = abs(array)\n",
      "    elif labels.startswith('F-obs-filtered'):\n",
      "      fobs = array\n",
      "  if (None in [flags, fobs, fmodel]):\n",
      "    raise RuntimeError('Not a valid phenix.refine output file')\n",
      "  scores = get_r_free_flags_scores([flags], None)\n",
      "  test_flag_value = scores.test_flag_values[0]\n",
      "  flags = flags.customized_copy(data=flags.data()==test_flag_value)\n",
      "\n",
      "  (d_centers, \n",
      "   r_work_list, \n",
      "   r_free_list, \n",
      "   cc_work_list, \n",
      "   cc_free_list) = compute_r_factors(fobs, fmodel, flags)\n",
      "  plot_r_factors(d_centers, r_work_list, r_free_list)\n",
      "  plot_cc(d_centers, cc_work_list, cc_free_list)\n",
      "\n",
      "\n",
      "if (__name__ == '__main__'):\n",
      "  run(input_mtz='28molrepEdited_5_refine_001.mtz')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "842711a96848"
      },
      "source": [
        "##plotFcalcsrResolutionBin\n",
        "\n",
        "Example of computing Fcalcs and then plotting them by resolution bin. This script uses miller arrays and binner.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "3781f6c22bb4"
      },
      "source": [
      "\"\"\"\n",
      "This script reads in a phenix.refine mtz file.\n",
      "It plots the R-factor by resolution bin.\n",
      "The plots are made with matplotlib using miller arrays.\n",
      "It also plots the correlation coefficients.\n",
      "The plots were made with matplotlib.\n",
      "\n",
      "This script was adapted from an example script in iotbx:  \n",
      "\n",
      "Source:  https://github.com/cctbx/cctbx_project/blob/master/\n",
      "iotbx/examples/recalculate_phenix_refine_r_factors.py\n",
      "\"\"\"\n",
      "\n",
      "\n",
      "# get_ipython().run_line_magic('matplotlib', 'inline')\n",
      "\n",
      "from __future__ import absolute_import, division, print_function\n",
      "from iotbx.reflection_file_utils import get_r_free_flags_scores\n",
      "from iotbx.file_reader import any_file\n",
      "import matplotlib\n",
      "import matplotlib.pyplot as plt\n",
      "\n",
      "\n",
      "\n",
      "def compute_r_factors(fobs, fmodel, flags):\n",
      "  fmodel, fobs = fmodel.common_sets(other=fobs)\n",
      "  fmodel, flags = fmodel.common_sets(other=flags)\n",
      "  fc_work = fmodel.select(~(flags.data()))\n",
      "  fo_work = fobs.select(~(flags.data()))\n",
      "  fc_test = fmodel.select(flags.data())\n",
      "  fo_test = fobs.select(flags.data())\n",
      "  r_work = fo_work.r1_factor(fc_work)\n",
      "  r_free = fo_test.r1_factor(fc_test)\n",
      "  \n",
      "  print('r_work = %.4f' % r_work)\n",
      "  print('r_free = %.4f' % r_free)\n",
      "  print('')\n",
      "\n",
      "  binner = flags.setup_binner(n_bins=20)\n",
      "  d_star_power = 1.618034\n",
      "  centers = binner.bin_centers(d_star_power)\n",
      "  d_centers = list(centers**(-1 / d_star_power))\n",
      "#   for i in d_centers:\n",
      "#     print(i)\n",
      "    \n",
      "  fo_work.use_binning_of(flags)\n",
      "  fc_work.use_binner_of(fo_work)\n",
      "  fo_test.use_binning_of(fo_work)\n",
      "  fc_test.use_binning_of(fo_work)\n",
      "\n",
      "  r_work_list = []\n",
      "  r_free_list = []\n",
      "  cc_work_list = []\n",
      "  cc_free_list = []\n",
      "  for i_bin in fo_work.binner().range_all():\n",
      "    sel_work = fo_work.binner().selection(i_bin)\n",
      "    sel_test = fo_test.binner().selection(i_bin)\n",
      "    fo_work_bin = fo_work.select(sel_work)\n",
      "    fc_work_bin = fc_work.select(sel_work)\n",
      "    fo_test_bin = fo_test.select(sel_test)\n",
      "    fc_test_bin = fc_test.select(sel_test)\n",
      "    if fc_test_bin.size() == 0 : continue\n",
      "        \n",
      "    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_work_list.append(r_work_bin)\n",
      "    \n",
      "    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_free_list.append(r_free_bin)\n",
      "    \n",
      "    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()\n",
      "    cc_work_list.append(cc_work_bin)\n",
      "    \n",
      "    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()\n",
      "    cc_free_list.append(cc_free_bin)\n",
      "    \n",
      "    legend = flags.binner().bin_legend(i_bin, show_counts=False)\n",
      "    print('%s  %8d %8d  %.4f %.4f  %.3f %.3f' % (legend, fo_work_bin.size(),\n",
      "      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))\n",
      "    \n",
      "  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list\n",
      "\n",
      "\n",
      "def plot_r_factors(d_centers, r_work_list, r_free_list):\n",
      "  plt.scatter(d_centers, r_work_list, label=r'$\mathit{R_{work$')\n",
      "  plt.scatter(d_centers, r_free_list, label=r'$\mathit{R_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'R-factor (%)')\n",
      "  plt.legend(loc='upper right')\n",
      "  plt.savefig('Rs.pdf')\n",
      "  plt.close()\n",
      "\n",
      "\n",
      "def plot_cc(d_centers, cc_work_list, cc_free_list):\n",
      "  plt.scatter(d_centers, cc_work_list, label=r'$\mathit{CC_{work$')\n",
      "  plt.scatter(d_centers, cc_free_list, label=r'$\mathit{CC_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'Correlation Coefficeint Fo vs Fc (%)')\n",
      "  plt.legend(loc='lower right')\n",
      "  plt.savefig('CCs.pdf')\n",
      "\n",
      "\n",
      "def run(input_mtz):\n",
      "  mtz_in = any_file(input_mtz)\n",
      "  ma = mtz_in.file_server.miller_arrays\n",
      "  flags = fmodel = fobs = None\n",
      "  # select the output arrays from phenix.refine.  This could easily be modified\n",
      "  # to handle MTZ files from other programs.\n",
      "  for array in ma :\n",
      "    labels = array.info().label_string()\n",
      "    if labels.startswith('R-free-flags'):\n",
      "      flags = array\n",
      "    elif labels.startswith('F-model'):\n",
      "      fmodel = abs(array)\n",
      "    elif labels.startswith('F-obs-filtered'):\n",
      "      fobs = array\n",
      "  if (None in [flags, fobs, fmodel]):\n",
      "    raise RuntimeError('Not a valid phenix.refine output file')\n",
      "  scores = get_r_free_flags_scores([flags], None)\n",
      "  test_flag_value = scores.test_flag_values[0]\n",
      "  flags = flags.customized_copy(data=flags.data()==test_flag_value)\n",
      "\n",
      "  (d_centers, \n",
      "   r_work_list, \n",
      "   r_free_list, \n",
      "   cc_work_list, \n",
      "   cc_free_list) = compute_r_factors(fobs, fmodel, flags)\n",
      "  plot_r_factors(d_centers, r_work_list, r_free_list)\n",
      "  plot_cc(d_centers, cc_work_list, cc_free_list)\n",
      "\n",
      "\n",
      "if (__name__ == '__main__'):\n",
      "  run(input_mtz='28molrepEdited_5_refine_001.mtz')"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ecf739fabe8e"
      },
      "source": [
        "##plotIntensityResolutionBin\n",
        "\n",
        "Miller arrays to plot of bin mean intensity over dstar\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7e4b83db7929"
      },
      "source": [
      "from iotbx.file_reader import any_file\n",
      "import matplotlib.pyplot as plt\n",
      "\n",
      "f = any_file('/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz')\n",
      "\n",
      "print(f.file_type)\n",
      "f.show_summary()\n",
      "miller_arrays = f.file_server.miller_arrays\n",
      "iobs =  miller_arrays[3]\n",
      "flags = miller_arrays[0]\n",
      "iobs, flags = iobs.common_sets(other=flags)\n",
      "iobsData = iobs.data()\n",
      "list(iobsData[100:110])\n",
      "iobs.show_comprehensive_summary()\n",
      "# iobs.binner()\n",
      "n_bins = 20\n",
      "binner = iobs.setup_binner(n_bins=n_bins)\n",
      "binner.show_summary()\n",
      "\n",
      "used = list(binner.range_used())\n",
      "selections = [binner.selection(i) for i in used]\n",
      "means = [iobs.select(sel).mean() for sel in selections]\n",
      "\n",
      "from math import log\n",
      "lnmeans = [log(y) for y in means]\n",
      "\n",
      "d_star_power = 1.618034\n",
      "centers = binner.bin_centers(d_star_power)\n",
      "d_centers = list(centers**(-1 / d_star_power))\n",
      "d_centers\n",
      "\n",
      "# plt.ylabel('Natural log of the amplitudes squared')\n",
      "# plt.xlabel(r'$\textrm{d^*$ in $\textrm{\AA$')\n",
      "# ax.set_xlim(35, 1.5)\n",
      "# plt.scatter(d_centers,lnmeanss)\n",
      "\n",
      "fig, ax = plt.subplots()\n",
      "ax.scatter(d_centers,lnmeans)\n",
      "ax.set_xlim(8, 1.5)  # decreasing\n",
      "ax.set_xlabel(r'$d^*$ in $\AA$')\n",
      "ax.set_ylabel('Natural log of the intensities')\n",
      "ax.grid(False)\n",
      "plt.savefig('iobsvsdstar.pdf')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d28b7fea9e96"
      },
      "source": [
        "##cns2mtz\n",
        "\n",
        "Miller arrays to convert CNS reflection file into an mtz file\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "84d73e9e3fb"
      },
      "source": [
      "from iotbx import reflection_file_reader\n",
      "import os\n",
      "reflection_file = reflection_file_reader.any_reflection_file(file_name=os.path.expandvars('\$CNS_SOLVE/doc/html/tutorial/data/pen/scale.hkl'))\n",
      "from cctbx import crystal\n",
      "crystal_symmetry = crystal.symmetry( unit_cell=(97.37, 46.64, 65.47, 90, 115.4, 90), space_group_symbol='C2')\n",
      "miller_arrays = reflection_file.as_miller_arrays( crystal_symmetry=crystal_symmetry)\n",
      "mtz_dataset = None\n",
      "for miller_array in miller_arrays:\n",
      "    if (mtz_dataset is None):\n",
      "        mtz_dataset = miller_array.as_mtz_dataset(\n",
      "            column_root_label=miller_array.info().labels[0]) \n",
      "    else:\n",
      "        mtz_dataset.add_miller_array(\n",
      "            miller_array=miller_array, \n",
      "            column_root_label=miller_array.info().labels[0])\n",
      "mtz_object = mtz_dataset.mtz_object() \n",
      "mtz_object.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "85890d5642"
      },
      "source": [
        "##FsigmaFbyabsLindex\n",
        "\n",
        "L-plot\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "90eacbf06431"
      },
      "source": [
      "import pandas as pd\n",
      "\"\"\"\n",
      "Use pandas to read in a hkl file with whitespace separators into a dataframe.\n",
      "Append to the dataframe a column with F/sigmaF values.\n",
      "Append to the dataframe a column with the absolute value of the L indices.\n",
      "Average F/sigmaF by absL index.\n",
      "Write to absL and F/sigmaF to csv file.\n",
      "\"\"\"\n",
      "mtzdata = pd.read_csv('1524start.hkl', names=['H','K','L','F','SIGF'], sep='\s+')\n",
      "mtzdata['FovSigF'] = mtzdata.apply(lambda row: row['F'] / row['SIGF'], axis=1)\n",
      "mtzdata['absL'] = mtzdata.apply(lambda row: abs(row['L']), axis=1)\n",
      "FovSigFabsL = mtzdata.groupby([mtzdata.absL]).FovSigF.mean()\n",
      "FovSigFabsL.to_csv('test2.csv')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "bc9f349322ff"
      },
      "source": [
        "##changeMtzColumns\n",
        "\n",
        "Read in mtz file and write out with fewer columns.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "81e30e9303d5"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_in = any_reflection_file('/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz')\n",
      "\n",
      "miller_arrays = hkl_in.as_miller_arrays()\n",
      "\n",
      "i_obs =  miller_arrays[3]\n",
      "r_free_flags = miller_arrays[0]\n",
      "f_obs = i_obs.f_sq_as_f()\n",
      "\n",
      "mtz_dataset = i_obs.as_mtz_dataset(column_root_label='I')\n",
      "mtz_dataset.add_miller_array(f_obs, column_root_label='F')\n",
      "mtz_dataset.add_miller_array(r_free_flags,column_root_label='FreeR_flag')\n",
      "mtz_dataset.mtz_object().write('loxodata.mtz')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "5d4f3d9968d9"
      },
      "source": [
        "##normalizedStructureFactors\n",
        "\n",
        "Calculate quasi-normalized structure factor.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "f58e6bf1d1f6"
      },
      "source": [
      "all_e_values = miller_array.quasi_normalize_structure_factors().sort(by_value='data')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d9520a027af8"
      },
      "source": [
        "##readMtzFile\n",
        "\n",
        "Read in a mtz file into a Miller array with iotbx.file_reader.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5cf164535014"
      },
      "source": [
      "from iotbx.file_reader import any_file\n",
      "mtz_in = any_file('data.mtz', force_type='mtz')\n",
      "miller_arrays = mtz_in.file_server.miller_arrays\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "3ae10ec4b8ec"
      },
      "source": [
        "##pattersonContourMap\n",
        "\n",
        "Read in a mtz file with iotbx.file_reader.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "ab1ad444f02d"
      },
      "source": [
      "import numpy\n",
      "from matplotlib import pyplot\n",
      "import gemmi\n",
      "# https://gemmi.readthedocs.io/en/latest/grid.html\n",
      "ccp4 = gemmi.read_ccp4_map('/Users/blaine/4bqrPatterson.ccp4')\n",
      "ccp4.setup()\n",
      "arr = numpy.array(ccp4.grid, copy=False)\n",
      "x = numpy.linspace(0, ccp4.grid.unit_cell.a, num=arr.shape[0], endpoint=False)\n",
      "y = numpy.linspace(0, ccp4.grid.unit_cell.b, num=arr.shape[1], endpoint=False)\n",
      "X, Y = numpy.meshgrid(x, y, indexing='ij')\n",
      "pyplot.rcParams['figure.figsize'] = (8.0, 8.0)\n",
      "pyplot.contour(X, Y, arr[:,:,0],500, zorder=1,linestyles='solid')\n",
      "pyplot.gca().set_aspect('equal', adjustable='box')\n",
      "pyplot.show()\n",
      "\n",
      "arr2 = numpy.array(ccp4.grid, copy=False)\n",
      "x = numpy.linspace(0, ccp4.grid.unit_cell.a, num=arr2.shape[0], endpoint=False)\n",
      "z = numpy.linspace(0, ccp4.grid.unit_cell.c, num=arr2.shape[1], endpoint=False)\n",
      "X, Z = numpy.meshgrid(x, z, indexing='ij')\n",
      "pyplot.rcParams['figure.figsize'] = (4.0, 20.5)\n",
      "pyplot.contour(X, Z, arr[:,:,0],500, zorder=1, linestyles='solid')\n",
      "pyplot.gca().set_aspect('equal', adjustable='box')\n",
      "pyplot.savefig('patterson.png', dpi=600)\n",
      "pyplot.show()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "989223222ae3"
      },
      "source": [
        "##condaInstall2\n",
        "\n",
        "The conda commands to remove old env and create a new one for  cctbx. These commands need to be run on the command line.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "860c0a60b53b"
      },
      "source": [
      "conda remove --name cctbx37\n",
      "conda create -n cctbx38 -c conda-forge cctbx-base python=3.8\n",
      "conda activate cctbx38\n",
      "conda install -c conda-forge cctbx-base\n",
      "conda install -c anaconda ipykernel\n",
      "python -m ipykernel install --user --name cctbx38 --display-name 'cctbx38'\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "3e42466908d7"
      },
      "source": [
        "##eigenvalues\n",
        "\n",
        "The commands to find the eigenvalues and eigenvectors on a tensor. The code is from a post to cctbxbb on 10 December 2020 by Richard Gildea in a reply to Robert Oeffner about code in cctbx for finding eigenvalues and eigenvectors. Robert was requesting the analog in cctbx to scipy.linalg.eig.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "96acb1dfc4e9"
      },
      "source": [
      "from scitbx.array_family import flex;\n",
      "from scitbx.linalg import eigensystem;\n",
      "m = flex.double(($1:-2, -4, 2, -2, 1, 2, 4, 2, 5));\n",
      "m.reshape(flex.grid(3,3));\n",
      "es = eigensystem.real_symmetric(m);\n",
      "list(es.values());\n",
      "list(es.vectors());\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ec2822796e7e"
      },
      "source": [
        "##IpIm\n",
        "\n",
        "Scatter plot of I(+) and (I(-). The presence of an anomalous signal is indicated by deviations from x=y.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "1ce7381bcb5a"
      },
      "source": [
      "%matplotlib inline\n",
      "import matplotlib.pyplot as plt\n",
      "import matplotlib as mpl\n",
      "import matplotlib.ticker as ticker\n",
      "from matplotlib.ticker import MultipleLocator #, FormatStrFormatter\n",
      "from matplotlib.ticker import FuncFormatter\n",
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "\n",
      "# >>> change the mtz file name\n",
      "hkl_file = any_reflection_file('3hz7.mtz')\n",
      "miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)\n",
      "Iobs = miller_arrays[1]\n",
      "i_plus, i_minus = Iobs.hemispheres_acentrics()\n",
      "ipd = i_plus.data()\n",
      "ip=list(ipd)\n",
      "imd = i_minus.data()\n",
      "im = list(imd)\n",
      "len(im)\n",
      "\n",
      "comma_fmt = FuncFormatter(lambda x, p: format(int(x), ','))\n",
      "\n",
      "mpl.rcParams['savefig.dpi'] = 600\n",
      "mpl.rcParams['figure.dpi'] = 600\n",
      "\n",
      "# Set to width of a one column on a two-column page.\n",
      "# May want to adjust settings for a slide.\n",
      "fig, ax = plt.subplots(figsize=[3.25, 3.25])\n",
      "ax.scatter(ip,im,c='k',alpha=0.3,s=5.5)\n",
      "ax.set_xlabel(r'I(+)',fontsize=12)\n",
      "ax.set_ylabel(r'I(-)',fontsize=12)\n",
      "ax.xaxis.set_major_locator(MultipleLocator(50000.))\n",
      "ax.yaxis.set_major_locator(MultipleLocator(50000.))\n",
      "ax.get_xaxis().set_major_formatter(comma_fmt)\n",
      "ax.get_yaxis().set_major_formatter(comma_fmt)\n",
      "\n",
      "plt.xticks(fontsize=12)\n",
      "plt.yticks(fontsize=12)\n",
      "ax.grid(False)\n",
      "\n",
      "# >>> change name of the figure file\n",
      "plt.savefig('3hz7IpIm.pdf',bbox_inches='tight')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "386cebc0d23f"
      },
      "source": [
        "##fetchFASTA\n",
        "\n",
        "Fetch fasta file from RCSB.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c62430b8f9ec"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='pdb', mirror='rcsb', format='pdb', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "f8c336010ace"
      },
      "source": [
        "##fetchXrayCif\n",
        "\n",
        "Fetch X-ray data from RCSB in mmCIF format.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2e24ebeb70e4"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='xray', mirror='rcsb', format='cif', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "6a997f772167"
      },
      "source": [
        "##fetchAtomicCif\n",
        "\n",
        "Fetch  atomic coordinates from RCSB in mmCIF format.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c64679bf9438"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='xray', mirror='rcsb', format='cif', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ffbd9f1d1b39"
      },
      "source": [
        "##symmetryFromPDB\n",
        "\n",
        "Print the symmetry from a PDB file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "6ec0bd245579"
      },
      "source": [
      "import iotbx\n",
      "iotbx.pdb.crystal_symmetry_from_pdb.extract_from('3nd4.pdb')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "2b5bede9d72a"
      },
      "source": [
        "##makeMaps\n",
        "\n",
        "Read in mtz and pdb file and write map coefficients to a separate mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "765ba417d1a"
      },
      "source": [
      "from mmtbx.maps.utils import create_map_from_pdb_and_mtz\n",
      "\"\"\"The phenix.maps commandline tool is the recommended approach.\"\"\"\n",
      "id='3nd4'\n",
      "create_map_from_pdb_and_mtz(\n",
      "          pdb_file='%s.pdb' % id,\n",
      "          mtz_file='%s.mtz' % id,\n",
      "          output_file='%s_maps.mtz' % id,\n",
      "          fill=False,\n",
      "          out=None,\n",
      "          llg_map=False,\n",
      "          remove_unknown_scatterering_type=True,\n",
      "          assume_pdb_data=False)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8d7b25cca6bd"
      },
      "source": [
        "##testCCTBX\n",
        "\n",
        "Enter this snippet on the command line in an empty directory.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5025b65b968b"
      },
      "source": [
      "libtbx.run_tests_parallel module=libtbx module=cctbx nproc=6\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "bdd2e85881dc"
      },
      "source": [
        "##condaInstall4Ununtu\n",
        "\n",
        "Enter this snippet on the command line in an empty directory.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7a9e16ce67ad"
      },
      "source": [
      "apt install nodejs git\n",
      "wget https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh\n",
      "bash Anaconda3-2020.02-Linux-x86_64.sh\n",
      "conda create -n pc37 python=3.7 schrodinger::pymol-bundle=2.4.1 conda-forge::cctbx-base conda-forge::jupyter\n",
      "conda activate pc37\n",
      "conda install conda-forge::jupyterlab=2.2.0\n",
      "# The following may be needed\n",
      "# jupyter serverextension enable --py jupyterlab --user \n",
      "pip install jupyterlab-snippets-multimenus\n",
      "jupyter lab build\n",
      "# Might be needed\n",
      "# jupyter lab clean\n",
      "jupyter --path # select the top option under Data for storing the libraries\n",
      "cd ~/.local/share/jupyter # change as per output from prior line\n",
      "mkdir multimenus_snippets\n",
      "cd multimenus_snippets\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbx.git cctbx\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbxplus.git cctbx+\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnips.git pymol\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnipsplus.git pymol+\n",
      "jupyter lab # or libtbx.python -m jupyter-lab\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "12f0f34f0fc6"
      },
      "source": [
        "##condaListEnvs\n",
        "\n",
        "List the currently available envs. The bang (!) enables running this command in a Juptyer Notebook. Delete the bang to run the command in the terminal.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "e0e718d3add5"
      },
      "source": [
      "!conda env list"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8dc294d1c551"
      },
      "source": [
        "##condaRemoveEnv\n",
        "\n",
        "Remove a specified env. The bang (!) enables running this command in a Juptyer Notebook. Delete the bang to run the command in the terminal.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2973e5507ae"
      },
      "source": [
      "!conda env remove --name cctbx37"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a1c44c620cd4"
      },
      "source": [
        "##condaInstallCCTBXInsidePyMOL\n",
        "\n",
        "Install cctbx inside PyMOL. This protocol worked on a Mac OS.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7fa5ddf0995f"
      },
      "source": [
      "# At the PyMOL prompt in the PyMOL GUI paste the following:\n",
      "conda install conda-forge::cctbx-base conda-forge::jupyter \n",
      "conda install conda-forge::jupyterlab=2.2.0\n",
      "conda install conda-forge::cctbx-base\n",
      "# In a terminal\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter serverextension enable --py jupyterlab --user\n",
      "/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol --user\n",
      "/Applications/PyMOL.app/Contents/bin/pip install jupyterlab-snippets-multimenus\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter lab build\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter lab clean\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter --path # select the top option under Data for storing the libraries\n",
      "cd ~.local/share/jupyter # change as per output from prior line\n",
      "mkdir multimenus_snippets\n",
      "cd multimenus_snippets\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbx.git cctbx\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbxplus.git cctbx+\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnips.git pymol\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnipsplus.git pymol+\n",
      "/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol37 --user\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e6249ac28ba5"
      },
      "source": [
        "##testCCTBXpython38\n",
        "\n",
        "Bash script to test new install with Python3.8.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "e69fe490e96a"
      },
      "source": [
      "#! /bin/sh\n",
      "set noglob\n",
      "set verbose\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/libtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/iotbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/boost_adptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/fable/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/scitbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cctbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cctbx/run_examples.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/smtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cflib_adaptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/reduce/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/mmtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cctbx_website/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/gltbx/run_tests.py"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "237c957869a3"
      },
      "source": [
        "##testCCTBXpython37\n",
        "\n",
        "Bash script to test new install with Python3.7.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "a42c7f496aef"
      },
      "source": [
      "#! /bin/sh\n",
      "set noglob\n",
      "set verbose\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/libtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/iotbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/boost_adptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/fable/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/scitbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cctbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cctbx/run_examples.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/smtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cflib_adaptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/reduce/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/mmtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cctbx_website/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/gltbx/run_tests.py"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "7281b577ec1a"
      },
      "source": [
        "##gemmiInstall\n",
        "\n",
        "Command to install gemmi (https://github.com/project-gemmi/gemmi). Documentation is located here (https://gemmi.readthedocs.io/en/latest/install.html).\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "a1f79818c823"
      },
      "source": [
      "# Install from PyPi\n",
      "pip install gemmi\n",
      "# OR install the latest version directly from github.\n",
      "pip install git+https://github.com/project-gemmi/gemmi.git"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "9d46fef916ee"
      },
      "source": [
        "##fixPermissionsOnMacSDK\n",
        "\n",
        "Fix permission on MacOS SDK. Adjust the version number of the sdk as needed.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "1b5789a397a5"
      },
      "source": [
      "cd /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/\n",
      "sudo ln -s MacOSX.sdk MacOSX10.15.sdk"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "9d82fad75465"
      },
      "source": [
        "##averageB\n",
        "\n",
        "Print the average B factor for all atoms.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "1fa867fe9512"
      },
      "source": [
      "from iotbx import pdb\n",
      "pdb_name = \"~/Downloads/4RBQ.pdb\"\n",
      "pdb_inp = pdb.input(file_name=pdb_name)\n",
      "atoms = pdb_inp.atoms()\n",
      "bfactors = atoms.extract_b()\n",
      "bfactors.format_mean('%6.2f Ang.^2')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8dfe45f4a043"
      },
      "source": [
        "##lastsnip\n",
        "\n",
        "This is a place holdhoder snippet because the last snippet in the list cannot have a comma after it.\n",
        "\n",
        "*PyMOL*"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "208168398ac9"
      },
      "source": [
        "##millerMtzImports\n",
        "\n",
        "Read a mtz file into a miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "62d3cc6895be"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_file = any_reflection_file('3hz7.mtz')\n",
      "miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8b4b6d33116b"
      },
      "source": [
        "##millerArrayLabels\n",
        "\n",
        "Print column labels in a Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "83a47537bccf"
      },
      "source": [
      "[print('Miller Array %s: %s' % (i, miller_array.info().labels)) for i, miller_array in list(enumerate(miller_arrays))[:2]]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "bc12eecc7a0"
      },
      "source": [
        "##millerArrayWavelengths\n",
        "\n",
        "Print wavelengths of each miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d2be19c12c18"
      },
      "source": [
      "[print('Miller Array %s: %s' % (i, miller_array.info().wavelength)) for i, miller_array in list(enumerate(miller_arrays))]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "268edb6e0128"
      },
      "source": [
        "##millerArraySources\n",
        "\n",
        "Print the source of each miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "166a05057972"
      },
      "source": [
      "print('Miller Array %s: %s' % (i, miller_array.info().source)) for i, miller_array in list(enumerate(miller_arrays))]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e0ca4156699c"
      },
      "source": [
        "##millerArrayLengths\n",
        "\n",
        "Print length of miller arrays (i.e., the number of datasets in a mtz file).\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "faae0e7066a6"
      },
      "source": [
      "len(miller_arrays)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "84aefb22c1aa"
      },
      "source": [
        "##millerArraySymmetry\n",
        "\n",
        "Print the crystal symmetry of each miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d823a99d4213"
      },
      "source": [
      "[print('Miller Array %s: %s' % (i, miller_array.info().crystal_symmetry_from_file)) for i, miller_array in list(enumerate(miller_arrays))]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e37b51f35931"
      },
      "source": [
        "##millerArrayHKLs\n",
        "\n",
        "Print all of the miller indices for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "68be1765ce64"
      },
      "source": [
      "[print(hkl) for hkl in miller_arrays[0].indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "fc8428423e53"
      },
      "source": [
        "##millerArrayMethods\n",
        "\n",
        "Print the available methods for the Miller class.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "ec890efbd83f"
      },
      "source": [
      "dir(miller_arrays[0])\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a4a93b6b0a8c"
      },
      "source": [
        "##millerArraydstar\n",
        "\n",
        "Return the resolution range in d* in a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "41c40922a9d3"
      },
      "source": [
      "miller_arrays[0].min_max_d_star_sq()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c8053e6577ef"
      },
      "source": [
        "##millerArrayDminDmax\n",
        "\n",
        "Return the resolution range in Angstroms for a Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "9a5fce31c3a3"
      },
      "source": [
      "miller_arrays[0].d_max_min()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "893e5da08192"
      },
      "source": [
        "##millerArrayIsigma\n",
        "\n",
        "Return the I/sig overall for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "f3fa3a7fe014"
      },
      "source": [
      "miller_arrays[0].i_over_sig_i()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "5caf7351d4"
      },
      "source": [
        "##millerArrayCC\n",
        "\n",
        "Return CC one-half sigma tau for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "401407a262d2"
      },
      "source": [
      "miller_arrays[0].cc_one_half_sigma_tau()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "192482022257"
      },
      "source": [
        "##millerArrayCConeHalf\n",
        "\n",
        "Return CC one-half for a given Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "cf41e7e951f0"
      },
      "source": [
      "miller_arrays[0].cc_one_half()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "6a03c2489db6"
      },
      "source": [
        "##millerArrayBijvoetRatios\n",
        "\n",
        "Print the Bijvoet ratios in a specified Miller array.  May have to average by bin first.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "350f95424ead"
      },
      "source": [
      "[print(i) for i in miller_arrays[0].bijvoet_ratios()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d85fc84cb1a8"
      },
      "source": [
        "##millerArrayMeasurability\n",
        "\n",
        "Return the `measurability` of the anomalous signal in a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "6b27fa87c72"
      },
      "source": [
      "miller_arrays[0].measurability()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "35b3248a77eb"
      },
      "source": [
        "##millerArrayAnomalousSignal\n",
        "\n",
        "Return the anomalous signal in a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "17755cd1bfb1"
      },
      "source": [
      "miller_arrays[0].anomalous_signal()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a41d1efbd2f8"
      },
      "source": [
        "##millerArrayComprehensiveSummary\n",
        "\n",
        "Show comprehensive summary for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "a47b4a02b5e6"
      },
      "source": [
      "miller_arrays[0].show_comprehensive_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e5f479dcd50b"
      },
      "source": [
        "##millerArrayCountBijvoetPairs\n",
        "\n",
        "Show number of bijvoet pairs for a specified Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "841f26aed1c5"
      },
      "source": [
      "miller_arrays[0].n_bijvoet_pairs()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a47efa001cf6"
      },
      "source": [
        "##millerArrayWilsonRatio\n",
        "\n",
        "Show wilson ratio of miller array for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "b088ca566917"
      },
      "source": [
      "miller_arrays[0].wilson_ratio()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "3c8d4fed4bc7"
      },
      "source": [
        "##millerArrayUnpackIpIn\n",
        "\n",
        "Unpack into I(+) and I(-) for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "446eb66d531f"
      },
      "source": [
      "Iobs = miller_arrays[0]\n",
      "i_plus, i_minus = Iobs.hemispheres_acentrics()\n",
      "ipd = i_plus.data()\n",
      "ip=list(ipd)\n",
      "imd = i_minus.data()\n",
      "im = list(imd)\n",
      "len(im)\n",
      "Iobs.show_summary()\n",
      "print(Iobs.info())\n",
      "print(Iobs.observation_type())\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4df81701ca5d"
      },
      "source": [
        "##millerArrayPrintSelectRows\n",
        "\n",
        "Print five rows of the Iobs for a specified Miller array. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2ca578e6442c"
      },
      "source": [
      "list(Iobs[100:105])\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "34b8af1bc1de"
      },
      "source": [
        "##millerArrayExtractIntensities\n",
        "\n",
        "Extract just the intensities for a give Miller array and print ten rows of them.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "ae86a88233df"
      },
      "source": [
      "Iobs = miller_arrays[0]\n",
      "iobsdata = Iobs.data()\n",
      "list(iobsdata[100:110])\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ea29c0120f25"
      },
      "source": [
        "##millerArrayPrintntensities\n",
        "\n",
        "Print all of the intensities for a given Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "b7244e71c098"
      },
      "source": [
      "[print(hkl) for hkl in miller_arrays[1].data()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8c2af4eb6bd"
      },
      "source": [
        "##millerArrayconvert2mtz\n",
        "\n",
        "Convert the miller array into a mtz_dataset and write out as a mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "89c5a531b296"
      },
      "source": [
      "# Convert the miller array into a mtz_dataset and write out as a mtz file.\n",
      "mtz_dataset = Iobs.as_mtz_dataset(column_root_label='I')\n",
      "mtz_dataset.mtz_object().write('3hz7intensities.mtz')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c96ef65d7e72"
      },
      "source": [
        "##millerArrayReadMtz\n",
        "\n",
        "Read in the mtz file and print its column labels as a sanity check.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "caf513cc7bbb"
      },
      "source": [
      "mtz_filename2 = '3hz7intensities.mtz'\n",
      "mtz_file2 = mtz.object(mtz_filename2)\n",
      "mtz_file2.column_labels()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "96248422e5c7"
      },
      "source": [
        "##millerArray2Dictionary\n",
        "\n",
        "Set up the arrays as dictionaries\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "b1e1e45d0b04"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='3nd4.mtz')\n",
      "# Only works with mtz.object. \n",
      "# Does not work if mtz is read in with iotbx.file_reader.\n",
      "miller_arrays_dict = mtz_obj.as_miller_arrays_dict()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "35cce1a44022"
      },
      "source": [
        "##millerArrayDictionaryKeys\n",
        "\n",
        "Print the miller keys() of a miller dictionary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "48d1178aeef7"
      },
      "source": [
      "miller_arrays_dict.keys()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "2c7b88499527"
      },
      "source": [
        "##millerArrayDictPrintColumns\n",
        "\n",
        "Print the column labels of Miller dictionary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d40ed1e99264"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='/Users/blaine/3nd4.mtz')\n",
      "# Only works with mtz.object. Does not work if mtz is read in with iotbx.file_reader.\n",
      "miller_arrays_dict = mtz_obj.as_miller_arrays_dict()\n",
      "[print(f'Column label: {key[2]')  for key in miller_arrays_dict.keys()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "55632319e8e9"
      },
      "source": [
        "##millerArrayBuildFromUnitCell\n",
        "\n",
        "Peter Zwart's code for generating the indices for unit cell of given symmetry and dimensions. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "56b644a89b93"
      },
      "source": [
      "from cctbx import miller\n",
      "import cctbx\n",
      "from cctbx import crystal\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='Fd-3m',\n",
      "        unit_cell=(5.4307,5.4307,5.4307,90.00,90.0,90.00)),\n",
      "    anomalous_flag=True,\n",
      "    d_min=0.4)\n",
      "\n",
      "[print(hkl) for hkl in ms.indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "b58a3485f66e"
      },
      "source": [
        "##millerArrayMapToASU\n",
        "\n",
        "Map generated reflections to the asu and print.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "886d75ddeae8"
      },
      "source": [
      "from cctbx import miller\n",
      "import cctbx\n",
      "from cctbx import crystal\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='Fd-3m',\n",
      "        unit_cell=(5.4307,5.4307,5.4307,90.00,90.0,90.00)),\n",
      "    anomalous_flag=True,\n",
      "    d_min=0.4)\n",
      "\n",
      "msu = ms.map_to_asu()\n",
      "[print(hkl2) for hkl2 in msu.indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4ffecd39637d"
      },
      "source": [
        "##millerArrayPrintSummary\n",
        "\n",
        "Read mtz file into a miller array and print summary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "4d9212194b1f"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_in = any_reflection_file(file_name='3nd4.mtz')\n",
      "miller_arrays = hkl_in.as_miller_arrays()\n",
      "f_obs = miller_arrays[0]\n",
      "f_obs.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c53d3e16d746"
      },
      "source": [
        "##mtzObjectSummary\n",
        "\n",
        "Read mtz file into a mtz object and print summary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "9e0a15c8c416"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='/Users/blaine/3nd4.mtz')\n",
      "mtz_obj.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ff1d6fecee92"
      },
      "source": [
        "##millerArrayFromMtz\n",
        "\n",
        "Read mtz file into a Miller array.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c33a55f687e"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='3nd4.mtz')\n",
      "miller_arrays = mtz_obj.as_miller_arrays()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "52396db4729f"
      },
      "source": [
        "##millerArrayTruncate\n",
        "\n",
        "Read mtz file into a Miller array, truncate, and print summary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "dddbc23278dc"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='3nd4.mtz')\n",
      "miller_arrays = mtz_obj.as_miller_arrays()\n",
      "miller_array_truncated = miller_arrays[0].resolution_filter(d_min=2, d_max=5)\n",
      "print(miller_array_truncated)\n",
      "miller_array_truncated.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "1b333a322347"
      },
      "source": [
        "##millerArrayDictColumnLabels\n",
        "\n",
        "Print column labels of a Miller array dictionary.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5ea895b2ecdd"
      },
      "source": [
      "[print(f'Column label: {key[2]')  for key in miller_arrays_dict.keys()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "78decee0abad"
      },
      "source": [
        "##condaInstall\n",
        "\n",
        "The conda commands to install cctbx with the jupyter notebook, pandas, and xarray.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "927349abbab4"
      },
      "source": [
      "conda create --name cctbx37 python=3.7\n",
      "conda activate cctbx37\n",
      "conda install -c conda-forge cctbx-base jupyter pandas xarray\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "6e130c16ddb2"
      },
      "source": [
        "##condaRemove\n",
        "\n",
        "The conda commands to remove cctbx37 env.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2dcbcb90617e"
      },
      "source": [
      "conda remove --name cctbx37"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d24ce6413d08"
      },
      "source": [
        "##fetchPDB\n",
        "\n",
        "Fetch pdb file from RCSB in PDB format.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "74491a5d3935"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='pdb', mirror='rcsb', format='pdb', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "1615ebe8d95f"
      },
      "source": [
        "##updateCLT\n",
        "\n",
        "Update the command line tools for Xcode on Mac OS X. \n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d44b18ba1982"
      },
      "source": [
      "sudo touch /tmp/.com.apple.dt.CommandLineTools.installondemand.in-progress\n",
      "softwareupdate -l\n",
      "# Update command line tools via software update.\n",
      "sudo rm /tmp/.com.apple.dt.CommandLineTools.installondemand.in-progress"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "fdcb6594ccdd"
      },
      "source": [
        "##extractCrystalSymmetry\n",
        "\n",
        "Extract crystal symmetry from mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c8132818a43b"
      },
      "source": [
      "from __future__ import absolute_import, division, print_function\n",
      "from iotbx import mtz\n",
      "from cctbx import crystal\n",
      "\n",
      "def extract_from(file_name):\n",
      "  mtz_object = mtz.object(file_name=file_name)\n",
      "  assert mtz_object.n_symmetry_matrices() > 0\n",
      "  return mtz_object.crystals()[0].crystal_symmetry()\n",
      "  \n",
      " extract_from(file_name='3nd4.mtz')\n",
      " $0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a442e31a37e5"
      },
      "source": [
        "##plotDstarsLogMeans\n",
        "\n",
        "Generate the list of dstars and logMeans as lists for plotting by matplotlib.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5e0a6360d081"
      },
      "source": [
      "\"\"\"Generate the list of dstars and logMeans as lists\n",
      "for plotting by matplotlib.\"\"\"\n",
      "\n",
      "used = list(binner.range_used())\n",
      "selections = [binner.selection(i) for i in used]\n",
      "\n",
      "# make means of the intensities by bin\n",
      "means = [Iobs.select(sel).mean() for sel in selections]\n",
      "from math import log\n",
      "lnmeans = [log(y) for y in means]\n",
      "\n",
      "# meansBR = [Iobs.bijvoet_ratios().select(sel).mean() for sel in selections]\n",
      "\n",
      "# make d_centers\n",
      "d_star_power = 1.618034\n",
      "centers = binner.bin_centers(d_star_power)\n",
      "d_centers = list(centers**(-1 / d_star_power))\n",
      "\n",
      "%matplotlib inline\n",
      "import matplotlib.pyplot as plt\n",
      "import matplotlib as mpl\n",
      "mpl.rcParams['savefig.dpi'] = 600\n",
      "mpl.rcParams['figure.dpi'] = 600\n",
      "\n",
      "fig, ax = plt.subplots(figsize=[3.25, 2.])\n",
      "ax.scatter(d_centers,lnmeans,c='k',alpha=0.3,s=5.5)\n",
      "\n",
      "ax.set_xlim(8, 1.5) # decreasing time\n",
      "ax.set_xlabel(r'$d^*$ in $\AA$',fontsize=12)\n",
      "ax.set_ylabel('ln(I)',fontsize=12)\n",
      "plt.xticks(fontsize=12)\n",
      "plt.yticks(fontsize=12)\n",
      "ax.grid(False)\n",
      "plt.savefig('3hz7iobsvsdstar.pdf',bbox_inches='tight')\n",
      "plt.show()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "c4a620720987"
      },
      "source": [
        "##plotDstarsMeasurability\n",
        "\n",
        "Generate the list of dstars and measurability as lists for plotting by matplotlib.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "29208760aa3a"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_file = any_reflection_file('3hz7.mtz')\n",
      "miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)\n",
      "\n",
      "Iobs = miller_arrays[1]\n",
      "# Set up the bins\n",
      "n_bins = 50\n",
      "binner = Iobs.setup_binner(n_bins=n_bins)\n",
      "# binner.show_summary()\n",
      "used = list(binner.range_used())\n",
      "selections = [binner.selection(i) for i in used]\n",
      "\n",
      "# make d_centers for the x-axis\n",
      "d_star_power = 1.618034\n",
      "centers = binner.bin_centers(d_star_power)\n",
      "d_centers = list(centers**(-1 / d_star_power))\n",
      "\n",
      "# make list of the measurabilities by resolution bin\n",
      "meas = [Iobs.select(sel).measurability() for sel in selections]\n",
      "\n",
      "%matplotlib inline\n",
      "import matplotlib.pyplot as plt\n",
      "import matplotlib as mpl\n",
      "mpl.rcParams['savefig.dpi'] = 600\n",
      "mpl.rcParams['figure.dpi'] = 600\n",
      "\n",
      "fig, ax = plt.subplots(figsize=[3.25, 2.])\n",
      "ax.scatter(d_centers,lnmeans,c='k',alpha=0.3,s=5.5)\n",
      "\n",
      "ax.set_xlim(8, 1.5) # decreasing time\n",
      "ax.set_xlabel(r'$d^*$ in $\AA$',fontsize=12)\n",
      "ax.set_ylabel('ln(I)',fontsize=12)\n",
      "plt.xticks(fontsize=12)\n",
      "plt.yticks(fontsize=12)\n",
      "ax.grid(False)\n",
      "plt.savefig('3hz7measureability.pdf',bbox_inches='tight')\n",
      "plt.show()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "aadb5199ff9a"
      },
      "source": [
        "##computeAllMillerIndices\n",
        "\n",
        "Compute all possible Miller indices.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "bfcc5fc9bc1"
      },
      "source": [
      "from cctbx import miller\n",
      "\n",
      "def generate_reflection_indices(uc, dmin):\n",
      "    maxh, maxk, maxl = uc.max_miller_indices(dmin)\n",
      "\n",
      "    indices = []\n",
      "\n",
      "    for h in range(-maxh, maxh + 1):\n",
      "        for k in range(-maxk, maxk + 1):\n",
      "            for l in range(-maxl, maxl + 1):\n",
      "                if h == 0 and k == 0 and l == 0:\n",
      "                    continue\n",
      "                if uc.d((h, k, l)) < dmin:\n",
      "                    continue\n",
      "        indices.append((h, k, l))\n",
      "    return indices\n",
      "    \n",
      "uc=(5.4307,5.4307,5.4307,90.00,90.0,90.00)\n",
      "dmin=1.0\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4fc0ba1f78a2"
      },
      "source": [
        "##computeAllMillerIndicesASU\n",
        "\n",
        "Compute all possible Miller indices in the ASU.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "76e2cde17500"
      },
      "source": [
      "from cctbx import miller\n",
      "import cctbx\n",
      "from cctbx import crystal\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='Fd-3m',\n",
      "        unit_cell=('5.4307,5.4307,5.4307,90.00,90.0,90.00') ),\n",
      "    anomalous_flag=False,\n",
      "    d_min=0.4)\n",
      "\n",
      "for hkl in ms.indices():\n",
      "    print(hkl)\n",
      "\n",
      "# map the reflections to the asu and print\n",
      "  \n",
      "msu = ms.map_to_asu()\n",
      "[print(hkl2) for hkl2 in msu.indices()]\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "39af28d53c59"
      },
      "source": [
        "##computeAllMillerIndicesUnitCell\n",
        "\n",
        "Build miller indices given unit cell and resolution limit.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7b53fc216787"
      },
      "source": [
      "from cctbx import crystal\n",
      "from cctbx import miller\n",
      "\n",
      "ms = miller.build_set(\n",
      "    crystal_symmetry=crystal.symmetry(\n",
      "        space_group_symbol='P4',\n",
      "        unit_cell=(150.8,150.8,250.4,90.0,90.0,90.0)),\n",
      "    anomalous_flag=False,\n",
      "    d_min=1.4)\n",
      "msu = ms.map_to_asu()\n",
      "[print(hkl) for hkl in msu.indices()]\n",
      "print(msu.show_comprehensive_summary())\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "4dac871a712d"
      },
      "source": [
        "##extractReflectionMtzFile\n",
        "\n",
        "Extract the reflections from a mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "d9e90a70be34"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz')\n",
      "mtz_obj.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "b986eaa9b6ec"
      },
      "source": [
        "##extractReflectionInShell\n",
        "\n",
        "Extract the reflections in a shell.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "cf8e647451e0"
      },
      "source": [
      "from iotbx import mtz\n",
      "mtz_obj = mtz.object(file_name='2V89.mtz')\n",
      "miller_arrays = mtz_obj.as_miller_arrays()\n",
      "for miller_array in miller_arrays:\n",
      "    miller_array_truncated = miller_array.resolution_filter(d_min=2, d_max=5)\n",
      "print(miller_array_truncated)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a9e08c396c23"
      },
      "source": [
        "##plotRfactorResolutionBin\n",
        "\n",
        "Read in a phenix.refine mtz file. It plots the work and free R-factors by resolution bin.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "956cb3fcc790"
      },
      "source": [
      "#!/usr/bin/env python\n",
      "# coding: utf-8\n",
      "\"\"\" \n",
      "This script reads in a phenix.refine mtz file.\n",
      "It plots the R-factor by resolution bin.\n",
      "The plots are made with matplotlib using miller arrays.\n",
      "It also plots the correlation coefficients.\n",
      "The plots were made with matplotlib.\n",
      "\n",
      "This script was adapted from an example script in iotbx:  \n",
      "\n",
      "Source:  https://github.com/cctbx/cctbx_project/blob/master/\n",
      "iotbx/examples/recalculate_phenix_refine_r_factors.py\n",
      "\"\"\"\n",
      "\n",
      "\n",
      "# get_ipython().run_line_magic('matplotlib', 'inline')\n",
      "\n",
      "from __future__ import absolute_import, division, print_function\n",
      "from iotbx.reflection_file_utils import get_r_free_flags_scores\n",
      "from iotbx.file_reader import any_file\n",
      "import matplotlib\n",
      "import matplotlib.pyplot as plt\n",
      "\n",
      "\n",
      "\n",
      "def compute_r_factors(fobs, fmodel, flags):\n",
      "  fmodel, fobs = fmodel.common_sets(other=fobs)\n",
      "  fmodel, flags = fmodel.common_sets(other=flags)\n",
      "  fc_work = fmodel.select(~(flags.data()))\n",
      "  fo_work = fobs.select(~(flags.data()))\n",
      "  fc_test = fmodel.select(flags.data())\n",
      "  fo_test = fobs.select(flags.data())\n",
      "  r_work = fo_work.r1_factor(fc_work)\n",
      "  r_free = fo_test.r1_factor(fc_test)\n",
      "  \n",
      "  print('r_work = %.4f' % r_work)\n",
      "  print('r_free = %.4f' % r_free)\n",
      "  print('')\n",
      "\n",
      "  binner = flags.setup_binner(n_bins=20)\n",
      "  d_star_power = 1.618034\n",
      "  centers = binner.bin_centers(d_star_power)\n",
      "  d_centers = list(centers**(-1 / d_star_power))\n",
      "#   for i in d_centers:\n",
      "#     print(i)\n",
      "    \n",
      "  fo_work.use_binning_of(flags)\n",
      "  fc_work.use_binner_of(fo_work)\n",
      "  fo_test.use_binning_of(fo_work)\n",
      "  fc_test.use_binning_of(fo_work)\n",
      "\n",
      "  r_work_list = []\n",
      "  r_free_list = []\n",
      "  cc_work_list = []\n",
      "  cc_free_list = []\n",
      "  for i_bin in fo_work.binner().range_all():\n",
      "    sel_work = fo_work.binner().selection(i_bin)\n",
      "    sel_test = fo_test.binner().selection(i_bin)\n",
      "    fo_work_bin = fo_work.select(sel_work)\n",
      "    fc_work_bin = fc_work.select(sel_work)\n",
      "    fo_test_bin = fo_test.select(sel_test)\n",
      "    fc_test_bin = fc_test.select(sel_test)\n",
      "    if fc_test_bin.size() == 0 : continue\n",
      "        \n",
      "    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_work_list.append(r_work_bin)\n",
      "    \n",
      "    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_free_list.append(r_free_bin)\n",
      "    \n",
      "    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()\n",
      "    cc_work_list.append(cc_work_bin)\n",
      "    \n",
      "    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()\n",
      "    cc_free_list.append(cc_free_bin)\n",
      "    \n",
      "    legend = flags.binner().bin_legend(i_bin, show_counts=False)\n",
      "    print('%s  %8d %8d  %.4f %.4f  %.3f %.3f' % (legend, fo_work_bin.size(),\n",
      "      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))\n",
      "    \n",
      "  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list\n",
      "\n",
      "\n",
      "def plot_r_factors(d_centers, r_work_list, r_free_list):\n",
      "  plt.scatter(d_centers, r_work_list, label=r'$\mathit{R_{work$')\n",
      "  plt.scatter(d_centers, r_free_list, label=r'$\mathit{R_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'R-factor (%)')\n",
      "  plt.legend(loc='upper right')\n",
      "  plt.savefig('Rs.pdf')\n",
      "  plt.close()\n",
      "\n",
      "\n",
      "def plot_cc(d_centers, cc_work_list, cc_free_list):\n",
      "  plt.scatter(d_centers, cc_work_list, label=r'$\mathit{CC_{work$')\n",
      "  plt.scatter(d_centers, cc_free_list, label=r'$\mathit{CC_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'Correlation Coefficeint Fo vs Fc (%)')\n",
      "  plt.legend(loc='lower right')\n",
      "  plt.savefig('CCs.pdf')\n",
      "\n",
      "\n",
      "def run(input_mtz):\n",
      "  mtz_in = any_file(input_mtz)\n",
      "  ma = mtz_in.file_server.miller_arrays\n",
      "  flags = fmodel = fobs = None\n",
      "  # select the output arrays from phenix.refine.  This could easily be modified\n",
      "  # to handle MTZ files from other programs.\n",
      "  for array in ma :\n",
      "    labels = array.info().label_string()\n",
      "    if labels.startswith('R-free-flags'):\n",
      "      flags = array\n",
      "    elif labels.startswith('F-model'):\n",
      "      fmodel = abs(array)\n",
      "    elif labels.startswith('F-obs-filtered'):\n",
      "      fobs = array\n",
      "  if (None in [flags, fobs, fmodel]):\n",
      "    raise RuntimeError('Not a valid phenix.refine output file')\n",
      "  scores = get_r_free_flags_scores([flags], None)\n",
      "  test_flag_value = scores.test_flag_values[0]\n",
      "  flags = flags.customized_copy(data=flags.data()==test_flag_value)\n",
      "\n",
      "  (d_centers, \n",
      "   r_work_list, \n",
      "   r_free_list, \n",
      "   cc_work_list, \n",
      "   cc_free_list) = compute_r_factors(fobs, fmodel, flags)\n",
      "  plot_r_factors(d_centers, r_work_list, r_free_list)\n",
      "  plot_cc(d_centers, cc_work_list, cc_free_list)\n",
      "\n",
      "\n",
      "if (__name__ == '__main__'):\n",
      "  run(input_mtz='28molrepEdited_5_refine_001.mtz')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "842711a96848"
      },
      "source": [
        "##plotFcalcsrResolutionBin\n",
        "\n",
        "Example of computing Fcalcs and then plotting them by resolution bin. This script uses miller arrays and binner.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "3781f6c22bb4"
      },
      "source": [
      "\"\"\"\n",
      "This script reads in a phenix.refine mtz file.\n",
      "It plots the R-factor by resolution bin.\n",
      "The plots are made with matplotlib using miller arrays.\n",
      "It also plots the correlation coefficients.\n",
      "The plots were made with matplotlib.\n",
      "\n",
      "This script was adapted from an example script in iotbx:  \n",
      "\n",
      "Source:  https://github.com/cctbx/cctbx_project/blob/master/\n",
      "iotbx/examples/recalculate_phenix_refine_r_factors.py\n",
      "\"\"\"\n",
      "\n",
      "\n",
      "# get_ipython().run_line_magic('matplotlib', 'inline')\n",
      "\n",
      "from __future__ import absolute_import, division, print_function\n",
      "from iotbx.reflection_file_utils import get_r_free_flags_scores\n",
      "from iotbx.file_reader import any_file\n",
      "import matplotlib\n",
      "import matplotlib.pyplot as plt\n",
      "\n",
      "\n",
      "\n",
      "def compute_r_factors(fobs, fmodel, flags):\n",
      "  fmodel, fobs = fmodel.common_sets(other=fobs)\n",
      "  fmodel, flags = fmodel.common_sets(other=flags)\n",
      "  fc_work = fmodel.select(~(flags.data()))\n",
      "  fo_work = fobs.select(~(flags.data()))\n",
      "  fc_test = fmodel.select(flags.data())\n",
      "  fo_test = fobs.select(flags.data())\n",
      "  r_work = fo_work.r1_factor(fc_work)\n",
      "  r_free = fo_test.r1_factor(fc_test)\n",
      "  \n",
      "  print('r_work = %.4f' % r_work)\n",
      "  print('r_free = %.4f' % r_free)\n",
      "  print('')\n",
      "\n",
      "  binner = flags.setup_binner(n_bins=20)\n",
      "  d_star_power = 1.618034\n",
      "  centers = binner.bin_centers(d_star_power)\n",
      "  d_centers = list(centers**(-1 / d_star_power))\n",
      "#   for i in d_centers:\n",
      "#     print(i)\n",
      "    \n",
      "  fo_work.use_binning_of(flags)\n",
      "  fc_work.use_binner_of(fo_work)\n",
      "  fo_test.use_binning_of(fo_work)\n",
      "  fc_test.use_binning_of(fo_work)\n",
      "\n",
      "  r_work_list = []\n",
      "  r_free_list = []\n",
      "  cc_work_list = []\n",
      "  cc_free_list = []\n",
      "  for i_bin in fo_work.binner().range_all():\n",
      "    sel_work = fo_work.binner().selection(i_bin)\n",
      "    sel_test = fo_test.binner().selection(i_bin)\n",
      "    fo_work_bin = fo_work.select(sel_work)\n",
      "    fc_work_bin = fc_work.select(sel_work)\n",
      "    fo_test_bin = fo_test.select(sel_test)\n",
      "    fc_test_bin = fc_test.select(sel_test)\n",
      "    if fc_test_bin.size() == 0 : continue\n",
      "        \n",
      "    r_work_bin = fo_work_bin.r1_factor(other=fc_work_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_work_list.append(r_work_bin)\n",
      "    \n",
      "    r_free_bin = fo_test_bin.r1_factor(other=fc_test_bin,\n",
      "      assume_index_matching=True)\n",
      "    r_free_list.append(r_free_bin)\n",
      "    \n",
      "    cc_work_bin = fo_work_bin.correlation(fc_work_bin).coefficient()\n",
      "    cc_work_list.append(cc_work_bin)\n",
      "    \n",
      "    cc_free_bin = fo_test_bin.correlation(fc_test_bin).coefficient()\n",
      "    cc_free_list.append(cc_free_bin)\n",
      "    \n",
      "    legend = flags.binner().bin_legend(i_bin, show_counts=False)\n",
      "    print('%s  %8d %8d  %.4f %.4f  %.3f %.3f' % (legend, fo_work_bin.size(),\n",
      "      fo_test_bin.size(), r_work_bin, r_free_bin, cc_work_bin, cc_free_bin))\n",
      "    \n",
      "  return d_centers, r_work_list, r_free_list, cc_work_list, cc_free_list\n",
      "\n",
      "\n",
      "def plot_r_factors(d_centers, r_work_list, r_free_list):\n",
      "  plt.scatter(d_centers, r_work_list, label=r'$\mathit{R_{work$')\n",
      "  plt.scatter(d_centers, r_free_list, label=r'$\mathit{R_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'R-factor (%)')\n",
      "  plt.legend(loc='upper right')\n",
      "  plt.savefig('Rs.pdf')\n",
      "  plt.close()\n",
      "\n",
      "\n",
      "def plot_cc(d_centers, cc_work_list, cc_free_list):\n",
      "  plt.scatter(d_centers, cc_work_list, label=r'$\mathit{CC_{work$')\n",
      "  plt.scatter(d_centers, cc_free_list, label=r'$\mathit{CC_{free$')\n",
      "  plt.xlabel(r'Resolution ($\mathrm{\AA$)')\n",
      "  plt.ylabel(r'Correlation Coefficeint Fo vs Fc (%)')\n",
      "  plt.legend(loc='lower right')\n",
      "  plt.savefig('CCs.pdf')\n",
      "\n",
      "\n",
      "def run(input_mtz):\n",
      "  mtz_in = any_file(input_mtz)\n",
      "  ma = mtz_in.file_server.miller_arrays\n",
      "  flags = fmodel = fobs = None\n",
      "  # select the output arrays from phenix.refine.  This could easily be modified\n",
      "  # to handle MTZ files from other programs.\n",
      "  for array in ma :\n",
      "    labels = array.info().label_string()\n",
      "    if labels.startswith('R-free-flags'):\n",
      "      flags = array\n",
      "    elif labels.startswith('F-model'):\n",
      "      fmodel = abs(array)\n",
      "    elif labels.startswith('F-obs-filtered'):\n",
      "      fobs = array\n",
      "  if (None in [flags, fobs, fmodel]):\n",
      "    raise RuntimeError('Not a valid phenix.refine output file')\n",
      "  scores = get_r_free_flags_scores([flags], None)\n",
      "  test_flag_value = scores.test_flag_values[0]\n",
      "  flags = flags.customized_copy(data=flags.data()==test_flag_value)\n",
      "\n",
      "  (d_centers, \n",
      "   r_work_list, \n",
      "   r_free_list, \n",
      "   cc_work_list, \n",
      "   cc_free_list) = compute_r_factors(fobs, fmodel, flags)\n",
      "  plot_r_factors(d_centers, r_work_list, r_free_list)\n",
      "  plot_cc(d_centers, cc_work_list, cc_free_list)\n",
      "\n",
      "\n",
      "if (__name__ == '__main__'):\n",
      "  run(input_mtz='28molrepEdited_5_refine_001.mtz')"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ecf739fabe8e"
      },
      "source": [
        "##plotIntensityResolutionBin\n",
        "\n",
        "Miller arrays to plot of bin mean intensity over dstar\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7e4b83db7929"
      },
      "source": [
      "from iotbx.file_reader import any_file\n",
      "import matplotlib.pyplot as plt\n",
      "\n",
      "f = any_file('/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz')\n",
      "\n",
      "print(f.file_type)\n",
      "f.show_summary()\n",
      "miller_arrays = f.file_server.miller_arrays\n",
      "iobs =  miller_arrays[3]\n",
      "flags = miller_arrays[0]\n",
      "iobs, flags = iobs.common_sets(other=flags)\n",
      "iobsData = iobs.data()\n",
      "list(iobsData[100:110])\n",
      "iobs.show_comprehensive_summary()\n",
      "# iobs.binner()\n",
      "n_bins = 20\n",
      "binner = iobs.setup_binner(n_bins=n_bins)\n",
      "binner.show_summary()\n",
      "\n",
      "used = list(binner.range_used())\n",
      "selections = [binner.selection(i) for i in used]\n",
      "means = [iobs.select(sel).mean() for sel in selections]\n",
      "\n",
      "from math import log\n",
      "lnmeans = [log(y) for y in means]\n",
      "\n",
      "d_star_power = 1.618034\n",
      "centers = binner.bin_centers(d_star_power)\n",
      "d_centers = list(centers**(-1 / d_star_power))\n",
      "d_centers\n",
      "\n",
      "# plt.ylabel('Natural log of the amplitudes squared')\n",
      "# plt.xlabel(r'$\textrm{d^*$ in $\textrm{\AA$')\n",
      "# ax.set_xlim(35, 1.5)\n",
      "# plt.scatter(d_centers,lnmeanss)\n",
      "\n",
      "fig, ax = plt.subplots()\n",
      "ax.scatter(d_centers,lnmeans)\n",
      "ax.set_xlim(8, 1.5)  # decreasing\n",
      "ax.set_xlabel(r'$d^*$ in $\AA$')\n",
      "ax.set_ylabel('Natural log of the intensities')\n",
      "ax.grid(False)\n",
      "plt.savefig('iobsvsdstar.pdf')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d28b7fea9e96"
      },
      "source": [
        "##cns2mtz\n",
        "\n",
        "Miller arrays to convert CNS reflection file into an mtz file\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "84d73e9e3fb"
      },
      "source": [
      "from iotbx import reflection_file_reader\n",
      "import os\n",
      "reflection_file = reflection_file_reader.any_reflection_file(file_name=os.path.expandvars('\$CNS_SOLVE/doc/html/tutorial/data/pen/scale.hkl'))\n",
      "from cctbx import crystal\n",
      "crystal_symmetry = crystal.symmetry( unit_cell=(97.37, 46.64, 65.47, 90, 115.4, 90), space_group_symbol='C2')\n",
      "miller_arrays = reflection_file.as_miller_arrays( crystal_symmetry=crystal_symmetry)\n",
      "mtz_dataset = None\n",
      "for miller_array in miller_arrays:\n",
      "    if (mtz_dataset is None):\n",
      "        mtz_dataset = miller_array.as_mtz_dataset(\n",
      "            column_root_label=miller_array.info().labels[0]) \n",
      "    else:\n",
      "        mtz_dataset.add_miller_array(\n",
      "            miller_array=miller_array, \n",
      "            column_root_label=miller_array.info().labels[0])\n",
      "mtz_object = mtz_dataset.mtz_object() \n",
      "mtz_object.show_summary()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "85890d5642"
      },
      "source": [
        "##FsigmaFbyabsLindex\n",
        "\n",
        "L-plot\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "90eacbf06431"
      },
      "source": [
      "import pandas as pd\n",
      "\"\"\"\n",
      "Use pandas to read in a hkl file with whitespace separators into a dataframe.\n",
      "Append to the dataframe a column with F/sigmaF values.\n",
      "Append to the dataframe a column with the absolute value of the L indices.\n",
      "Average F/sigmaF by absL index.\n",
      "Write to absL and F/sigmaF to csv file.\n",
      "\"\"\"\n",
      "mtzdata = pd.read_csv('1524start.hkl', names=['H','K','L','F','SIGF'], sep='\s+')\n",
      "mtzdata['FovSigF'] = mtzdata.apply(lambda row: row['F'] / row['SIGF'], axis=1)\n",
      "mtzdata['absL'] = mtzdata.apply(lambda row: abs(row['L']), axis=1)\n",
      "FovSigFabsL = mtzdata.groupby([mtzdata.absL]).FovSigF.mean()\n",
      "FovSigFabsL.to_csv('test2.csv')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "bc9f349322ff"
      },
      "source": [
        "##changeMtzColumns\n",
        "\n",
        "Read in mtz file and write out with fewer columns.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "81e30e9303d5"
      },
      "source": [
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "hkl_in = any_reflection_file('/Users/blaine/manuscripts/RETkinaseLoxo/ret_blu.mtz')\n",
      "\n",
      "miller_arrays = hkl_in.as_miller_arrays()\n",
      "\n",
      "i_obs =  miller_arrays[3]\n",
      "r_free_flags = miller_arrays[0]\n",
      "f_obs = i_obs.f_sq_as_f()\n",
      "\n",
      "mtz_dataset = i_obs.as_mtz_dataset(column_root_label='I')\n",
      "mtz_dataset.add_miller_array(f_obs, column_root_label='F')\n",
      "mtz_dataset.add_miller_array(r_free_flags,column_root_label='FreeR_flag')\n",
      "mtz_dataset.mtz_object().write('loxodata.mtz')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "5d4f3d9968d9"
      },
      "source": [
        "##normalizedStructureFactors\n",
        "\n",
        "Calculate quasi-normalized structure factor.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "f58e6bf1d1f6"
      },
      "source": [
      "all_e_values = miller_array.quasi_normalize_structure_factors().sort(by_value='data')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "d9520a027af8"
      },
      "source": [
        "##readMtzFile\n",
        "\n",
        "Read in a mtz file into a Miller array with iotbx.file_reader.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5cf164535014"
      },
      "source": [
      "from iotbx.file_reader import any_file\n",
      "mtz_in = any_file('data.mtz', force_type='mtz')\n",
      "miller_arrays = mtz_in.file_server.miller_arrays\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "3ae10ec4b8ec"
      },
      "source": [
        "##pattersonContourMap\n",
        "\n",
        "Read in a mtz file with iotbx.file_reader.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "ab1ad444f02d"
      },
      "source": [
      "import numpy\n",
      "from matplotlib import pyplot\n",
      "import gemmi\n",
      "# https://gemmi.readthedocs.io/en/latest/grid.html\n",
      "ccp4 = gemmi.read_ccp4_map('/Users/blaine/4bqrPatterson.ccp4')\n",
      "ccp4.setup()\n",
      "arr = numpy.array(ccp4.grid, copy=False)\n",
      "x = numpy.linspace(0, ccp4.grid.unit_cell.a, num=arr.shape[0], endpoint=False)\n",
      "y = numpy.linspace(0, ccp4.grid.unit_cell.b, num=arr.shape[1], endpoint=False)\n",
      "X, Y = numpy.meshgrid(x, y, indexing='ij')\n",
      "pyplot.rcParams['figure.figsize'] = (8.0, 8.0)\n",
      "pyplot.contour(X, Y, arr[:,:,0],500, zorder=1,linestyles='solid')\n",
      "pyplot.gca().set_aspect('equal', adjustable='box')\n",
      "pyplot.show()\n",
      "\n",
      "arr2 = numpy.array(ccp4.grid, copy=False)\n",
      "x = numpy.linspace(0, ccp4.grid.unit_cell.a, num=arr2.shape[0], endpoint=False)\n",
      "z = numpy.linspace(0, ccp4.grid.unit_cell.c, num=arr2.shape[1], endpoint=False)\n",
      "X, Z = numpy.meshgrid(x, z, indexing='ij')\n",
      "pyplot.rcParams['figure.figsize'] = (4.0, 20.5)\n",
      "pyplot.contour(X, Z, arr[:,:,0],500, zorder=1, linestyles='solid')\n",
      "pyplot.gca().set_aspect('equal', adjustable='box')\n",
      "pyplot.savefig('patterson.png', dpi=600)\n",
      "pyplot.show()\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "989223222ae3"
      },
      "source": [
        "##condaInstall2\n",
        "\n",
        "The conda commands to remove old env and create a new one for  cctbx. These commands need to be run on the command line.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "860c0a60b53b"
      },
      "source": [
      "conda remove --name cctbx37\n",
      "conda create -n cctbx38 -c conda-forge cctbx-base python=3.8\n",
      "conda activate cctbx38\n",
      "conda install -c conda-forge cctbx-base\n",
      "conda install -c anaconda ipykernel\n",
      "python -m ipykernel install --user --name cctbx38 --display-name 'cctbx38'\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "3e42466908d7"
      },
      "source": [
        "##eigenvalues\n",
        "\n",
        "The commands to find the eigenvalues and eigenvectors on a tensor. The code is from a post to cctbxbb on 10 December 2020 by Richard Gildea in a reply to Robert Oeffner about code in cctbx for finding eigenvalues and eigenvectors. Robert was requesting the analog in cctbx to scipy.linalg.eig.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "96acb1dfc4e9"
      },
      "source": [
      "from scitbx.array_family import flex;\n",
      "from scitbx.linalg import eigensystem;\n",
      "m = flex.double(($1:-2, -4, 2, -2, 1, 2, 4, 2, 5));\n",
      "m.reshape(flex.grid(3,3));\n",
      "es = eigensystem.real_symmetric(m);\n",
      "list(es.values());\n",
      "list(es.vectors());\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ec2822796e7e"
      },
      "source": [
        "##IpIm\n",
        "\n",
        "Scatter plot of I(+) and (I(-). The presence of an anomalous signal is indicated by deviations from x=y.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "1ce7381bcb5a"
      },
      "source": [
      "%matplotlib inline\n",
      "import matplotlib.pyplot as plt\n",
      "import matplotlib as mpl\n",
      "import matplotlib.ticker as ticker\n",
      "from matplotlib.ticker import MultipleLocator #, FormatStrFormatter\n",
      "from matplotlib.ticker import FuncFormatter\n",
      "from iotbx.reflection_file_reader import any_reflection_file\n",
      "\n",
      "# >>> change the mtz file name\n",
      "hkl_file = any_reflection_file('3hz7.mtz')\n",
      "miller_arrays = hkl_file.as_miller_arrays(merge_equivalents=False)\n",
      "Iobs = miller_arrays[1]\n",
      "i_plus, i_minus = Iobs.hemispheres_acentrics()\n",
      "ipd = i_plus.data()\n",
      "ip=list(ipd)\n",
      "imd = i_minus.data()\n",
      "im = list(imd)\n",
      "len(im)\n",
      "\n",
      "comma_fmt = FuncFormatter(lambda x, p: format(int(x), ','))\n",
      "\n",
      "mpl.rcParams['savefig.dpi'] = 600\n",
      "mpl.rcParams['figure.dpi'] = 600\n",
      "\n",
      "# Set to width of a one column on a two-column page.\n",
      "# May want to adjust settings for a slide.\n",
      "fig, ax = plt.subplots(figsize=[3.25, 3.25])\n",
      "ax.scatter(ip,im,c='k',alpha=0.3,s=5.5)\n",
      "ax.set_xlabel(r'I(+)',fontsize=12)\n",
      "ax.set_ylabel(r'I(-)',fontsize=12)\n",
      "ax.xaxis.set_major_locator(MultipleLocator(50000.))\n",
      "ax.yaxis.set_major_locator(MultipleLocator(50000.))\n",
      "ax.get_xaxis().set_major_formatter(comma_fmt)\n",
      "ax.get_yaxis().set_major_formatter(comma_fmt)\n",
      "\n",
      "plt.xticks(fontsize=12)\n",
      "plt.yticks(fontsize=12)\n",
      "ax.grid(False)\n",
      "\n",
      "# >>> change name of the figure file\n",
      "plt.savefig('3hz7IpIm.pdf',bbox_inches='tight')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "386cebc0d23f"
      },
      "source": [
        "##fetchFASTA\n",
        "\n",
        "Fetch fasta file from RCSB.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c62430b8f9ec"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='pdb', mirror='rcsb', format='pdb', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "f8c336010ace"
      },
      "source": [
        "##fetchXrayCif\n",
        "\n",
        "Fetch X-ray data from RCSB in mmCIF format.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2e24ebeb70e4"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='xray', mirror='rcsb', format='cif', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "6a997f772167"
      },
      "source": [
        "##fetchAtomicCif\n",
        "\n",
        "Fetch  atomic coordinates from RCSB in mmCIF format.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "c64679bf9438"
      },
      "source": [
      "from iotbx.pdb.fetch import get_pdb\n",
      "import sys\n",
      "get_pdb(id='3nd4',data_type='xray', mirror='rcsb', format='cif', log=sys.stdout)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "ffbd9f1d1b39"
      },
      "source": [
        "##symmetryFromPDB\n",
        "\n",
        "Print the symmetry from a PDB file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "6ec0bd245579"
      },
      "source": [
      "import iotbx\n",
      "iotbx.pdb.crystal_symmetry_from_pdb.extract_from('3nd4.pdb')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "2b5bede9d72a"
      },
      "source": [
        "##makeMaps\n",
        "\n",
        "Read in mtz and pdb file and write map coefficients to a separate mtz file.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "765ba417d1a"
      },
      "source": [
      "from mmtbx.maps.utils import create_map_from_pdb_and_mtz\n",
      "\"\"\"The phenix.maps commandline tool is the recommended approach.\"\"\"\n",
      "id='3nd4'\n",
      "create_map_from_pdb_and_mtz(\n",
      "          pdb_file='%s.pdb' % id,\n",
      "          mtz_file='%s.mtz' % id,\n",
      "          output_file='%s_maps.mtz' % id,\n",
      "          fill=False,\n",
      "          out=None,\n",
      "          llg_map=False,\n",
      "          remove_unknown_scatterering_type=True,\n",
      "          assume_pdb_data=False)\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8d7b25cca6bd"
      },
      "source": [
        "##testCCTBX\n",
        "\n",
        "Enter this snippet on the command line in an empty directory.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "5025b65b968b"
      },
      "source": [
      "libtbx.run_tests_parallel module=libtbx module=cctbx nproc=6\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "bdd2e85881dc"
      },
      "source": [
        "##condaInstall4Ununtu\n",
        "\n",
        "Enter this snippet on the command line in an empty directory.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7a9e16ce67ad"
      },
      "source": [
      "apt install nodejs git\n",
      "wget https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh\n",
      "bash Anaconda3-2020.02-Linux-x86_64.sh\n",
      "conda create -n pc37 python=3.7 schrodinger::pymol-bundle=2.4.1 conda-forge::cctbx-base conda-forge::jupyter\n",
      "conda activate pc37\n",
      "conda install conda-forge::jupyterlab=2.2.0\n",
      "# The following may be needed\n",
      "# jupyter serverextension enable --py jupyterlab --user \n",
      "pip install jupyterlab-snippets-multimenus\n",
      "jupyter lab build\n",
      "# Might be needed\n",
      "# jupyter lab clean\n",
      "jupyter --path # select the top option under Data for storing the libraries\n",
      "cd ~/.local/share/jupyter # change as per output from prior line\n",
      "mkdir multimenus_snippets\n",
      "cd multimenus_snippets\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbx.git cctbx\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbxplus.git cctbx+\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnips.git pymol\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnipsplus.git pymol+\n",
      "jupyter lab # or libtbx.python -m jupyter-lab\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "12f0f34f0fc6"
      },
      "source": [
        "##condaListEnvs\n",
        "\n",
        "List the currently available envs. The bang (!) enables running this command in a Juptyer Notebook. Delete the bang to run the command in the terminal.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "e0e718d3add5"
      },
      "source": [
      "!conda env list"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8dc294d1c551"
      },
      "source": [
        "##condaRemoveEnv\n",
        "\n",
        "Remove a specified env. The bang (!) enables running this command in a Juptyer Notebook. Delete the bang to run the command in the terminal.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "2973e5507ae"
      },
      "source": [
      "!conda env remove --name cctbx37"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "a1c44c620cd4"
      },
      "source": [
        "##condaInstallCCTBXInsidePyMOL\n",
        "\n",
        "Install cctbx inside PyMOL. This protocol worked on a Mac OS.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "7fa5ddf0995f"
      },
      "source": [
      "# At the PyMOL prompt in the PyMOL GUI paste the following:\n",
      "conda install conda-forge::cctbx-base conda-forge::jupyter \n",
      "conda install conda-forge::jupyterlab=2.2.0\n",
      "conda install conda-forge::cctbx-base\n",
      "# In a terminal\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter serverextension enable --py jupyterlab --user\n",
      "/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol --user\n",
      "/Applications/PyMOL.app/Contents/bin/pip install jupyterlab-snippets-multimenus\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter lab build\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter lab clean\n",
      "/Applications/PyMOL.app/Contents/bin/jupyter --path # select the top option under Data for storing the libraries\n",
      "cd ~.local/share/jupyter # change as per output from prior line\n",
      "mkdir multimenus_snippets\n",
      "cd multimenus_snippets\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbx.git cctbx\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolcctbxplus.git cctbx+\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnips.git pymol\n",
      "git clone https://github.com/MooersLab/juptyerlabpymolpysnipsplus.git pymol+\n",
      "/Applications/PyMOL.app/Contents/bin/ipython kernel install --name pymol37 --user\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "e6249ac28ba5"
      },
      "source": [
        "##testCCTBXpython38\n",
        "\n",
        "Bash script to test new install with Python3.8.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "e69fe490e96a"
      },
      "source": [
      "#! /bin/sh\n",
      "set noglob\n",
      "set verbose\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/libtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/iotbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/boost_adptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/fable/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/scitbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cctbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cctbx/run_examples.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/smtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cflib_adaptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/reduce/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/mmtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/cctbx_website/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.8/site-packages/gltbx/run_tests.py"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "237c957869a3"
      },
      "source": [
        "##testCCTBXpython37\n",
        "\n",
        "Bash script to test new install with Python3.7.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "a42c7f496aef"
      },
      "source": [
      "#! /bin/sh\n",
      "set noglob\n",
      "set verbose\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/libtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/iotbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/boost_adptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/fable/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/scitbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cctbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cctbx/run_examples.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/smtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cflib_adaptbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/reduce/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/mmtbx/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/cctbx_website/run_tests.py\n",
      "libtbx.python $CONDA_PREFIX/lib/python3.7/site-packages/gltbx/run_tests.py"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "7281b577ec1a"
      },
      "source": [
        "##gemmiInstall\n",
        "\n",
        "Command to install gemmi (https://github.com/project-gemmi/gemmi). Documentation is located here (https://gemmi.readthedocs.io/en/latest/install.html).\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "a1f79818c823"
      },
      "source": [
      "# Install from PyPi\n",
      "pip install gemmi\n",
      "# OR install the latest version directly from github.\n",
      "pip install git+https://github.com/project-gemmi/gemmi.git"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "9d46fef916ee"
      },
      "source": [
        "##fixPermissionsOnMacSDK\n",
        "\n",
        "Fix permission on MacOS SDK. Adjust the version number of the sdk as needed.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "1b5789a397a5"
      },
      "source": [
      "cd /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/\n",
      "sudo ln -s MacOSX.sdk MacOSX10.15.sdk"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "9d82fad75465"
      },
      "source": [
        "##averageB\n",
        "\n",
        "Print the average B factor for all atoms.\n",
        "\n",
        "*Python*"
      ]
    },
    {
    "cell_type": "code",
    "metadata": {
      "id": "1fa867fe9512"
      },
      "source": [
      "from iotbx import pdb\n",
      "pdb_name = \"~/Downloads/4RBQ.pdb\"\n",
      "pdb_inp = pdb.input(file_name=pdb_name)\n",
      "atoms = pdb_inp.atoms()\n",
      "bfactors = atoms.extract_b()\n",
      "bfactors.format_mean('%6.2f Ang.^2')\n",
      "$0"
      ]
    },
    {
    "cell_type": "markdown",
    "metadata": {
      "id": "8dfe45f4a043"
      },
      "source": [
        "##lastsnip\n",
        "\n",
        "This is a place holdhoder snippet because the last snippet in the list cannot have a comma after it.\n",
        "\n",
        "*PyMOL*"
      ]
    }
  ]
}